In [155]:
import os
import json
import time
import subprocess
from google3.pyglib import gfile
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm,trange
from scipy import stats
from collections import defaultdict,Counter
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN,AgglomerativeClustering

import statistics
import uuid
import re
import copy

from saxml.client.python import sax

from google3.devtools import textminer as tm
from collections.abc import Collection, Mapping, Sequence
import dataclasses
import enum
import json
import textwrap
from typing import Optional, Tuple, Union
import urllib


from google3.learning.deepmind.evergreen.model_access.client.python import model_client
from google3.learning.deepmind.evergreen.model_access.client.python import content_api
from IPython import display

import colabtools
from colabtools import drive
from colabtools import auth


# meta description:
# this is the code for understanding and improving model planning ability
#       the current direction is to find meta-problem/actions through: (1) retrieve and decompose; (2) byte pair encoding
#       a scaling perspective: eventually we want the LLM to decompose the fuzzy relevant problems in the context and utilize the refined demos
# we are currently working on Azade's planning dataset
# the eval of the dataset requires external planning tools, e.g., VAL planner
# many of the auxiliary and comments are discussed in the comments
#     terminology:
#       design hoice: places where the are alternative designs
#       method: implemented design choice
#       finding: potential intermediate interpretation
#       limitation: discussion of what is missing in the current solutions

class SudoArgs:
  exist = None

DATA_DIR = "/cns/od-d/home/pagi/planning/training_data/"


# block_file = os.path.join(DATA_DIR, "planning/train_file_pddl_plan_3_20_v6.json")
# dev_block_file = os.path.join(DATA_DIR, "planning/dev_bw_3_9_v6.json")
# ood_dev = os.path.join(DATA_DIR, "planning/dev_file_ood_8_20_v6.json")

# train_file = os.path.join(DATA_DIR, "planning/training_file_pddl_plan_v6.json")
# dev_file = os.path.join(DATA_DIR, "planning/dev_file_pddl_plan_v6.json")
# dev_file = os.path.join(DATA_DIR, "planning/train_file_ood_8_20_v6.json")

# grid_file = os.path.join(DATA_DIR, "planning/training_grid_v2.json")
# logistics_file = os.path.join(DATA_DIR, "planning/training_logistics_v5.json")
# tetris_file = os.path.join(DATA_DIR, "planning/training_tetris_v1.json")
# trip_plan_jsonl = os.path.join(DATA_DIR, "planning/trip_planning.jsonl")

# train_file = os.path.join(DATA_DIR, "planning/training_logistics_v5_removedspace.json")
# dev_file = os.path.join(DATA_DIR, "planning/dev_logistics_v5_removedspace.json")

# train_file = os.path.join(DATA_DIR, "planning/training_logistics_v5.json")
# dev_file = os.path.join(DATA_DIR, "planning/dev_logistics_v5.json")

# train_file = os.path.join(DATA_DIR, "planning/training_grid_v2.json")
# dev_file = os.path.join(DATA_DIR, "planning/dev_grid_v2.json")
# # gfile.MkDir('/cns/od-d/home/xinranzhao/dev-300/')

train_file = os.path.join(DATA_DIR, "planning/training_tetris_v1.json")
dev_file = os.path.join(DATA_DIR, "planning/dev_tetris_v1.json")

In [156]:
# data loading
# plans_df = pd.read_json(block_file, lines=True, orient="records")
# test_df = pd.read_json(ood_dev, lines=True, orient="records")

plans_df = pd.read_json(train_file, lines=True, orient="records")
test_df = pd.read_json(dev_file, lines=True, orient="records")
# ood_df = pd.read_json(ood_dev, lines=True, orient="records")

with gfile.Open('/cns/od-d/home/xinranzhao/val_results/train_v6.json', 'r') as f:
  train_exec = json.load(f)

with gfile.Open('/cns/od-d/home/xinranzhao/val_results/dev_v6.json', 'r') as f:
  test_exec = json.load(f)

# with gfile.Open('/cns/od-d/home/xinranzhao/val_results/ood_v6.json', 'r') as f:
#   ood_exec = json.load(f)


# print('***total',len(plans_df))
# df = plans_df.iloc[:len(plans_df)]

# print('***total test',len(test_df))

# data_lines = plans_df.to_dict('records')
# print(len(data_lines))

def readable_plan_print(data_line_item,option="all"):
  # options: all, input, target
  if option in ["all","input"]:
    print("INPUT")
    for line in data_line_item["inputs"].split("\n"):
      print(line)
  if option in ["all","target"]:
    print("TARGET:")
    for line in data_line_item["targets"].split("\n"):
      print(line)

def readable_plan_print_compare(data_line_item1, data_line_item2):
  for key in ["inputs", "targets"]:
    print(key.upper())
    input1 = data_line_item1[key].split("\n")
    input2 = data_line_item2[key].split("\n")

    if len(input1) < len(input2):
      input2,input1 = input1, input2

    for i, line in enumerate(input1):
      if i < len(input2):
        print(line," "*(50-len(line)), input2[i])
      else:
        print(line)

# readable_plan_print(data_lines[0])

# readable_plan_print_compare(data_lines[0], data_lines[1])

# data_scope = 200

In [157]:
print(test_df.iloc[0]["targets"])
print(len(plans_df))
print(len(test_df))

(move_square f2-2f f3-2f square4)
(move_l_down f1-0f f2-0f f2-1f f3-0f f3-1f rightl0)
(move_l_down f2-0f f3-0f f3-1f f4-0f f4-1f rightl0)
(move_square f0-0f f0-1f square0)
(move_square f0-2f f0-3f square1)
(move_square f1-1f f2-1f square2)
(move_square f1-2f f0-2f square3)
(move_square f2-1f f3-1f square2)
(move_square f0-3f f1-3f square1)
(move_square f1-3f f2-3f square1)
(move_square f2-3f f3-3f square1)
(move_square f3-3f f4-3f square1)
(move_square f0-2f f0-3f square3)
(move_square f0-3f f1-3f square3)
(move_square f1-3f f2-3f square3)
(move_square f2-3f f3-3f square3)
(move_square f0-1f f0-0f square0)
(move_square f0-0f f1-0f square0)
(move_square f1-0f f2-0f square0)
(move_l_down f3-0f f4-0f f4-1f f5-0f f5-1f rightl0)
(move_square f2-0f f3-0f square0)
2463
600


## TM Usage Examples


In [158]:
# text minier pipeline record


# story = """
# abc
# """

# def wrap(s):  # print text wrapped at 80 columns
#   print('\n'.join(textwrap.wrap(s)))

#### TM pipeline #####

# d = tm.Document(story)
# s = d.summarize()
# wrap(s)

# embedder_classes = [
#     tm.models.RetsimEmbedder(),
#     tm.models.GeckoEmbedder(),
#     tm.models.UnivSentEnc(),
#     tm.models.UnivSentEncLarge(),
#     tm.models.ST5Embedder(),
#     tm.models.ST5Embedder_large(),
#     tm.models.ST5Embedder_3b(),
#     # tm.models.ST5Embedder_11b, # Failed copying input tensor to GPU error
# ]

# em = tm.models.UnivSentEnc() # small sent t5
# from the benchmark side: Geko > ST5 > Univ > Retsim
# em = tm.models.UnivSentEncLarge() # se
# em = tm.models.GeckoEmbedder() # distill from llm
# em = tm.models.ST5Embedder_3b()

# tm.embedding(["I like ice cream","are you ok"], embedding_model_override=em).shape # ndarray
# tm.set_model(tm.models.ULM_24b())
# em = tm.models.GeminiEmbedder()
# m = tm.models.ULM_24b()
# m = tm.models.ULM_340b()
# m = tm.models.Gemini()
# m = tm.models.SaxLLM("/sax/llmau/llmit_ulm_24b") #???
# m = tm.models.SaxLLM("/sax/evergreeb/gemini_m_1lmit")

# tm.set_embedding_model(tm.models.GenaiEmbedder())
# tm.embedding(["I like ice cream"])

# add constraints to Gemini:
# output_dict = {'answer': str, 'explanation': str, 'population': int}
# m.query(
#         q='What is the capital of the United Kingdom?',
#         constraint=output_dict
#         constraint='London|Paris'
        # temperature=temperature,
        # max_output_tokens=max_output_tokens,
        # stop_strings=stop_strings,
        # num_threads=num_threads,
        # top_p=top_p,
        # top_k=top_k,
        # timeout=timeout,
#     )
# m.query("What do you think of the number 11?",max_output_tokens=10)



##### AX API pipeline ######
# model = "/sax/llmau/llmit_ulm_24b"
# prompt = "What do you think of the number 11?"

# # With SAX, the default is to connect to the model directly. If you'd like to
# # use LM Proxy with sax, you must provide the LM Proxy blade address explicitly.
# options = sax.Options()
# options.proxy_addr = "sax.server.lm.lmservice-prod.blade.gslb.googleprod.com"

# model = sax.Model(model, options)

# # We are querying a language model.
# lm = model.LM()

# print(lm.Generate(prompt)[0][0])

# RQ0: what features in problems will lead to similar plan outputs

In [160]:
# in the test cases, we only see the problems, not the plans
# hypothetically, demonstrating similar plans as the output will be helpful

# what are the features for plan similarity
# 1. plans (we got here) are a series of actions where the orders matter, unlike the problems
# 2. plans can be partially similar, we can decompose them in the future
#         with the description of problem states, the decomposed subplans are essentially also a subproblem
# ToDo:
# 1. domain-aware plan simiarity
# 2. state-aware plan similarity (need per-step plan execution)
# 3. model-based similarity, what will be different with a model that is trained/untrained on planning


# print(plans_df)

# step 1: parse the plans and define the similarity of to plans

def plan_parse(target_list):
  parsed_plans = []
  for target in target_list:
    parsed_plans.append(target.split("\n"))
  return parsed_plans


def action_parse(parsed_plan,args):
  # return a list of action seqs from plans.
  # if not object_centric_parse: there will be only one seq per plan
  # else, there will be multiple seqs
  if args.object_centric_parse:
    action_dic = defaultdict(list)
    rel_map = defaultdict(list)

    for raw_action in parsed_plan:
      raw_action = raw_action.replace("(","").replace(")","")
      try:
        action = raw_action.split()[0]
      except:
        print(raw_action)
      objects = raw_action.split()[1:]
      if args.object_relevance_parse:
        for obj1 in objects:
          for obj2 in objects:
            if obj2 not in rel_map[obj1]:
              rel_map[obj1].append(obj2)

      for i, obj in enumerate(objects):
        if args.object_centric_affordance:
          # affordance only come with actions on >1 objects
          if len(objects) > 1:
          # use abstract object affordance, e.g., (stack b4 b6), b6 is with affordance notion _1
            action_dic[obj].append(action+"_"+str(i))
          if args.object_relevance_parse:
            for rel_obj in rel_map[obj]:
              action_dic[rel_obj].append(action+"_"+str(i)+"_rel")
        else:
          # ignore the affordance of the action
          action_dic[obj].append(action)
          if args.object_relevance_parse:
            for rel_obj in rel_map[obj]:
              action_dic[rel_obj].append(action+"_rel")
    return list(action_dic.values())

  else:
    action_list = []
    for raw_action in parsed_plan:
      action = "null"
      try:
        action = raw_action.split()[0]
      except:
        print(raw_action)

      action = action.replace("(","")
      action_list.append(action)
    return [action_list]


def convert_input_to_exec(inputs):
  output_seq = []
  status = 0
  for line in inputs.splitlines():
    if "goal" in line:
      break

    if status == 1:
      if len(line) > 2:
        output_seq.append("Adding " + line.strip())

    if "init" in line:
      status = 1

  return output_seq

def exec_output_parse(exec_outputs):
  parsed_plans = []
  for exec_output in exec_outputs:
    parsed_plan = []
    for line in exec_output.splitlines():
      if "Adding" in line or "Deleting" in line:
        parsed_plan.append(line)

    parsed_plans.append(parsed_plan)
  return parsed_plans

def execution_parse(parsed_plan,args):
  # return a list of action seqs from plans.
  # if not object_centric_parse: there will be only one seq per plan
  # else, there will be multiple seqs
  if args.object_centric_parse:
    action_dic = defaultdict(list)

    for raw_action in parsed_plan:
      segs = raw_action.split()
      meta_action = segs[0]
      action = segs[1].replace("(","").replace(")","")
      objects = segs[2:]
      objects = [obj.replace("(","").replace(")","") for obj in objects]
      if len(objects) == 0: #Adding (handempty)
        objects = ["hand"]

      for i, obj in enumerate(objects):

        if args.object_centric_affordance:
          # affordance only come with actions on >1 objects
          if len(objects) > 1:
          # use abstract object affordance, e.g., (stack b4 b6), b6 is with affordance notion _1
            action_dic[obj].append(meta_action+"_"+action+"_"+str(i))
          else:
            # ignore the affordance of the action
            action_dic[obj].append(meta_action+"_"+action)
        else:
          # no affordnace, just the meta action and action
          action_dic[obj].append(meta_action+"_"+action)

    return list(action_dic.values())

  else:
    action_list = []
    for raw_action in parsed_plan:
      segs = raw_action.split() # Deleting (ontable b1)
      meta_action = segs[0]
      action = raw_action.split()[1]
      action = action.replace("(","").replace(")","")
      action_list.append(meta_action+"_"+action)

    return [action_list]

# design choice: similarity method
# Method 1: longest common actions (extend longest common action+objects)
# Improvement:
#   1. repeated matching (May 28)
#   2. Object centric action parsing (May 29): how to include object but do not lose too much generalizability
#   3. Object relevance involved action parsing (May 29)
# limitations:
#   1.the provided plans need to be minimum sufficient -> a reduce module?
#   2.there might be multiple common action series in different order -> how BPE might help
#   3.there might be multiple occurences of small problems in the large problems
#   4.Action BLEU does not make sense since the uni-action similarity is useless

def longest_common_actions(action_list0, action_list1, args):
  m,n = len(action_list0), len(action_list1)
  # 2D DP table
  dp_table = [[0 for _ in range(n+1)] for _ in range(m+1)]
  end_m, end_n = 0,0
  max = 0
  for i in range(1,m+1):
    for j in range(1,n+1):
      if action_list0[i-1] == action_list1[j-1]:
        dp_table[i][j] = 1 + dp_table[i-1][j-1]
        if dp_table[i][j] > max:
          max = dp_table[i][j]
          end_m, end_n = i,j
      else:
        if args.consecutive:
          dp_table[i][j] = 0
        else:
          dp_table[i][j] = max(dp_table[i-1][j], dp_table[i][j-1])

  # restore the longest common list
  if args.consecutive:
    assert action_list1[end_n-max:end_n] == action_list0[end_m-max:end_m]
    return action_list1[end_n-max:end_n], [end_n-max,end_n]
  else:
    common_list = []
    i,j = m,n

    while i > 0 and j > 0:
      if action_list0[i-1] == action_list1[j-1]:
        common_list.append(action_list0[i-1])
        i = i - 1
        j = j - 1
      elif dp_table[i-1][j] > dp_table[i][j-1]:
        i = i - 1
      else:
        j = j - 1

    # also return the index to generate to serve as the interval for partial problems
    return common_list[::-1], [end_m-max,end_m]


def find_pair_sim_mapping_main(plans_df, test_df, train_exec, test_exec, args):
  pair_sim_mapping = []
  pair_sim_interval = []

  if not args.exec_based:
    parsed_plans = plan_parse(list(plans_df['targets']))
    test_plans = plan_parse(list(test_df['targets']))

    action_lists = [action_parse(parsed_plan,args) for parsed_plan in parsed_plans]
    test_action_lists = [action_parse(parsed_plan,args) for parsed_plan in test_plans]
  else:
    # execution based plans
    parsed_plans = exec_output_parse(list(train_exec))
    test_plans = exec_output_parse(list(test_exec))

    if args.init_exec:
      # we use the problems to initialize the plans
      train_inputs = list(plans_df['inputs'])
      test_inputs = list(test_df['inputs'])
      parse_plans = [convert_input_to_exec(input)+(parsed_plan) for input, parsed_plan in zip(train_inputs, parsed_plans)]
      test_plans = [convert_input_to_exec(input)+(test_plan) for input, test_plan in zip(test_inputs, test_plans)]

    action_lists = [execution_parse(parsed_plan,args) for parsed_plan in parsed_plans]
    test_action_lists = [execution_parse(parsed_plan,args) for parsed_plan in test_plans]


  # design choice: sampling 100 data points, 30.72 it/s required for each pair
  for i, action_list0 in enumerate(tqdm(test_action_lists[:args.test_scope])):
    this_pair_sim_mapping = []
    this_pair_sim_interval = []

    for j, action_list1 in enumerate(action_lists[:args.data_scope]):
      # if we assume a undirectioned metric
      if args.undir_metric and j < i:
        this_pair_sim_mapping.append(pair_sim_mapping[j][i])
        continue

      common_list_collection,common_len_collection = [],[]
      common_list = [] # dummy
      common_a1_interval = [0,0]
      max_common_len = 0

      for a0 in action_list0:
        this_common_list, this_common_len = [],[]
        for a1 in action_list1:
          common_list, common_a1_interval = longest_common_actions(a0, a1, args)
          common_len = len(common_list)
          # consider the case where there are multiple common_list
          if args.lca_with_reoccur:
            count0 = " ".join(a0).count(" ".join(common_list))
            count1 = " ".join(a1).count(" ".join(common_list))
            factor = max(count0, count1)
            common_len = factor * common_len

          # design choice: how and if we should penalize the length
          common_len = common_len**2 / (len(a0) * len(a1))
          if common_len > max_common_len:
            max_common_len = common_len

          this_common_list.append(common_list)
          this_common_len.append(common_len)

        common_list_collection.append(this_common_list)
        common_len_collection.append(this_common_len)

      if args.print_detail:
        print(len(common_list_collection))
        print(len(common_list_collection[0]))
        print(common_list_collection[0])

      # organize with mean(max(per object action series))
      sim_score = 0
      if args.score_conv == "mean-max":
        temp_score = [max(x) for x in common_len_collection]
        sim_score = sum(temp_score) / len(temp_score)

      if args.score_conv == "max-max":
        sim_score = max_common_len

      this_pair_sim_mapping.append(sim_score)
      this_pair_sim_interval.append(common_a1_interval)

    pair_sim_mapping.append(this_pair_sim_mapping)
    pair_sim_interval.append(this_pair_sim_interval)

  return pair_sim_mapping, pair_sim_interval

## Heatmap Gen

In [ ]:
def gen_heatmap_main(pair_sim_mapping, args):
  problem_size_annotation = []

  # note that
  for i in range(data_scope):
    problem_size_annotation.append(plans_df["inputs"][i].split("\n")[0].replace(")","").split("-")[-1])

  # print(len(problem_size_annotation))
  # print(problem_size_annotation)
  counter = defaultdict(int)
  for letter in problem_size_annotation:
    counter[letter] += 1

  x, y = [],[]
  for k,v in counter.items():
    x.append(k)
    y.append(v)

  index_order = list(np.argsort(y)[::-1])
  new_x = [x[i] for i in index_order]
  new_y = [y[i] for i in index_order]
  fig = plt.figure(figsize = (10, 5))
  plt.bar(new_x, new_y, color ='tab:blue', width = 0.4)
  plt.xlabel("Problem Size")
  plt.ylabel("# in the samples")

  int_keys = [int(x) for x in counter.keys()]
  size_ref = list(np.sort(int_keys))
  size_ref = [str(x) for x in size_ref]
  print(size_ref)

  list_of_lists = [[[] for x in range(len(size_ref))] for x in range(len(size_ref))]
  count_ref = [[0 for x in range(len(size_ref))] for x in range(len(size_ref))]

  for i, this_pair_sim in enumerate(pair_sim_mapping):
    for j, sim in enumerate(this_pair_sim):
      ind_first = size_ref.index(problem_size_annotation[i])
      ind_second = size_ref.index(problem_size_annotation[j])
      list_of_lists[ind_first][ind_second].append(sim)
      count_ref[ind_second][ind_first] += 1

  # calculate the average
  for i, this_list in enumerate(list_of_lists):
    for j, sim in enumerate(this_list):

      if args.heatmap == "mean":
        # average similarity
        list_of_lists[i][j] = sum(list_of_lists[i][j]) / count_ref[i][j]
      elif args.heatmap == "max":
        list_of_lists[i][j] = max(list_of_lists[i][j])


  df = pd.DataFrame()
  df["sizes"] = size_ref

  for i, p1_size in enumerate(size_ref):
      df[p1_size] = list_of_lists[i]
  df.set_index('sizes', drop=True, inplace=True)

  mask = np.triu(np.ones_like(df, dtype=bool))
  f, ax = plt.subplots(figsize=(9.5, 9))

  for i in range(len(size_ref)):
    mask[i][i] = False

  cmap = sns.diverging_palette(230, 20, as_cmap=True)

  sns.heatmap(df,annot=True,mask=mask, cmap='gist_gray_r', vmax=1, center=0.5,cbar=False,
      square=False, linewidths=.6, cbar_kws={"shrink": .5}, annot_kws={"fontsize":10})

  ax.set(ylabel=None)
  ax.tick_params(axis='x', labelsize= 15)
  ax.tick_params(axis='y', labelsize= 15)


data_scope = 400
test_scope = 400

args = SudoArgs()
args.data_scope = data_scope
args.test_scope = test_scope
args.print_detail = False # for debugging and visualization

args.consecutive = True # if not consecutive, we include the sublist
args.undir_metric = False # if the metric is undirection so that we can accelerate the compute
args.lca_with_reoccur = False # if we count re-occurence of the longest common actions

args.exec_based = True # use the execution of the plans, instead of the pure plans
args.init_exec = True # use the input as the initial condition of the plans

args.object_centric_parse = True # parse the actions by objects
args.object_centric_affordance = True # when parsing the actions, assume different locations with objects
args.object_relevance_parse = False # parse the actions by objects, add actions from relevant objects

args.score_conv = "mean-max" # max-max denotes  max(max(per object action series similarity))
args.score_conv_mode = "weighted" # if weighted, the overall common_len will be weighed by the common len of each
args.heatmap = "mean" # type of heatmap-

args.use_prompt = False # use prompt during embedding
args.plan_sim = "gecko" # token or embedding model names: [gecho,retsim,uni,unil,st5,st5l,st53b]
args.client = "goldfish_s" # m, xxs_it, m_it, m_it_bulk, xl_it, goldfish, goldfish_s, s_flash
args.num_demo = 1 # test: 0,1,4,8

args.icl_sim = "input" # use random, plan, or input similarity to sample the icl examples
args.icl_sim_mode = "static" # static, dynamic, if dynamic, we sample demostration with distribution, num_demo become the std offset

args.use_prompt = False
args.plan_sim = "emb" # embedding on token
args.heatmap = "mean" # type of heatmap


# pair_sim_mapping,pair_sim_interval = find_pair_sim_mapping_main(plans_df, plans_df,train_exec, test_exec, args)
# gen_heatmap_main(pair_sim_mapping, args)

# Finding 3: overview: high similarity, reducibility, shared object-wise actions
# example

# (1) from the mean similarity graph (overall, as previous),
      # small problems usually have higher overall similarity
      # we can observe that the similarity for 20-20 is similar to 20-13
# (2) from the max similarity graph (overall, as previous),
      # many problems can actually be reduced, size 9 case show that more data can result in a hit
      # even though there is no exact match, there are some problems that are very similar (~0.9)
      # question: what are the problems that can not be reduced? e.g., readable_plan_print_compare(data_lines[4], data_lines[8])
# (3) from mean similarity graph (object)
      # objects commonly share action sequences
      # larger block sizes have more shared object-wise actions, compared to small ones
# (4) from max similarity graph (object)
      # for large block size problems, in many cases there are shared object-wise acctions

## Input Similarity Measure

In [161]:
# Exp 2: correlation between plan similarity and surface
# from the example above, we can observe that similar plans may be from slightly different problems

# design choice: score the problem similarity with token match
# alternative, compute with embedding similarity

def convert_tm_embedding(data_lines, em, args, keyword="inputs"):
  texts = [line[keyword] for line in data_lines]

  if args.use_prompt:
    texts = [args.prompt+line[keyword] for line in data_lines]

  embs = tm.embedding(texts, embedding_model_override=em)
  return embs


def find_input_sim_main(plan_df, test_df, em, args):

  def input_token_match(item1, item2):
    # parse
    tokens1, tokens2 = [], []
    for line in item1.split("\n"):
      tokens1.extend(line.split())
    for line in item2.split("\n"):
      tokens2.extend(line.split())

    count = 0
    for token in tokens1:
      if token in tokens2:
        count += 1
    return count**2/(len(tokens1)*len(tokens2))


  pair_sim_mapping_problem = []

  data_lines = plan_df.to_dict(orient="records")
  test_lines = test_df.to_dict(orient="records")
  if args.plan_sim != "token":
    with gfile.Open('/cns/od-d/home/xinranzhao/pair-sim-maps/block_train_v6_inputs_embs.json', mode='r') as f:
      data_inputs_embs = json.load(f)

    with gfile.Open('/cns/od-d/home/xinranzhao/pair-sim-maps/block_dev_v6_inputs_embs.json', mode='r') as f:
      test_inputs_embs = json.load(f)

    embs = data_inputs_embs[:args.data_scope] #convert_tm_embedding(data_lines[:args.data_scope], em, args)
    test_embs = test_inputs_embs[:args.test_scope] # convert_tm_embedding(data_lines[:args.test_scope], em, args)

  for i, item1 in enumerate(tqdm(test_lines[:args.test_scope])):
    this_sim_mapping_problem = []
    for j, item2 in enumerate(data_lines[:args.data_scope]):

      if args.plan_sim == "token":
        # token sim
        sim = input_token_match(item1["inputs"], item2["inputs"])
      else:
        # cosine similarity
        sim = cosine_similarity([test_embs[i]], [embs[j]])[0][0]

      this_sim_mapping_problem.append(sim)

    pair_sim_mapping_problem.append(this_sim_mapping_problem)


  return pair_sim_mapping_problem

## Visualization

In [ ]:
# compute the macro-avg correlation accross items
# all_correlation = []
# for i, plan_sim in enumerate(pair_sim_mapping):
#   input_sim = pair_sim_mapping_problem[i]
#   res = stats.pearsonr(plan_sim, input_sim)[0]
#   all_correlation.append(res)

# # Finding 3: the correlation between plans and problems are not large, and sometimes negative from the word level
# # this motivates us to wonder how to capture the latent plan similarity from the problem similarity
# print(sum(all_correlation)/len(all_correlation))

# embedder_classes = [
#     tm.models.RetsimEmbedder(),
#     tm.models.GeckoEmbedder(),
#     tm.models.UnivSentEnc(),
#     tm.models.UnivSentEncLarge(),
#     tm.models.ST5Embedder(),
#     tm.models.ST5Embedder_large(),
#     tm.models.ST5Embedder_3b(),
#     # tm.models.ST5Embedder_11b, # Failed copying input tensor to GPU error
# ]

# prompt_template = """
# Please solve the problem:
# <input>
# Your plan as plain text without formatting:
# <plan>
# done.
# """


# demo_data_lines = []
# for i,line in enumerate(data_lines[:args.data_scope]):
#   if int(problem_size_annotation[i]) < 4:
#     demo_data_lines.append(line)

# em = tm.models.GeckoEmbedder()

# for k in range(5):
#   print("Demo:",k)
#   args.num_demo = k
#   args.prompt = "Please solve the problem:\n"

#   for i in range(args.num_demo):
#       args.prompt = prompt_template.replace("<input>",demo_data_lines[i]["inputs"]).replace("<plan>",demo_data_lines[i]["targets"]) +"\n"+ args.prompt

# pair_sim_mapping_problem, all_correlation = find_input_sim_main(plan_df, plan_df, em, args)

# Exp 1 Qualitative study:
# print(pair_sim_mapping)
# print(len(pair_sim_mapping[0]))

# find the max index besides itself
# max_ind_list = []
# score_1_pairs = []

# for i, this_pair_sim in enumerate(pair_sim_mapping):
#   max_score = 0
#   max_ind = 0
#   for j, score in enumerate(this_pair_sim):
#     if i == j:
#       continue
#     else:
#       if score > max_score:
#         max_score = score
#         max_ind = j

#     if score == 1.0:
#       if (j,i) not in score_1_pairs:
#         score_1_pairs.append((i, j))

#   max_ind_list.append(max_ind)

# -------------------------------------------- sep line -------------------------

# Finding 0: we can use these maxes to formulate a graph.
#       is the periodic distribution an data creation artifact?
# distribution of the cumulated
# cum_pair_sim = np.sum(pair_sim_mapping,axis=1)

# fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(14,4))

# x,y = [],[]
# for j, score in enumerate(cum_pair_sim):
#   x.append(j)
#   y.append(score)

# axs[0].bar(x, y, color ='tab:blue', width = 0.4)
# plt.sca(axs[0])
# plt.xlabel("Plan Problem Index")
# plt.ylabel("Cumulated Similarity")

# x,y,z = [],[],[]
# for i, data_line in enumerate(data_lines[:100]):
#   x.append(i)
#   y.append(len(data_line["targets"].split("\n")))
#   z.append(len(data_line["inputs"].split("\n")))

# axs[1].bar(x, y, color ='tab:blue', width = 0.4)
# plt.sca(axs[1])
# plt.xlabel("Plan Problem Index")
# plt.ylabel("# Target Lines")
# axs[2].bar(x, z, color ='tab:green', width = 0.4)
# plt.sca(axs[2])
# plt.xlabel("Plan Problem Index")
# plt.ylabel("# Input Lines")
# plt.show()
# # print(max_ind_list)

# -------------------------------------------- sep line -------------------------

# Finding 1: problem number 4 gots the highest most-similar count
# distribution of the most similar count
# count_occurrence = defaultdict(int)
# for ind in max_ind_list:
#   count_occurrence[ind] += 1

# x, y = [],[]
# for k,v in count_occurrence.items():
#   x.append(k)
#   y.append(v)

# index_order = list(np.argsort(y)[::-1])
# new_x = [x[i] for i in index_order]
# new_y = [y[i] for i in index_order]
# fig = plt.figure(figsize = (10, 5))
# plt.bar(new_x, new_y, color ='tab:blue', width = 0.4)

# readable_plan_print(data_lines[4])

# Finding 2: print some high similar cases, as expected, similar plans may be from slightly different problems
# [(4, 8), (4, 64), (8, 64), (13, 42), (22, 37), (22, 49), (22, 56), (22, 91),
# (37, 49), (37, 56), (37, 91), (49, 56), (49, 91), (55, 65), (55, 77), (56, 91), (65, 77)]

# print(score_1_pairs)


# -------------------------------------------- sep line -------------------------

# x,y = [],[]
# for j, score in enumerate(all_correlation):
#   x.append(j)
#   y.append(score)

# plt.bar(x, y, color ='tab:blue', width = 0.4)
# plt.xlabel("Plan Problem Index")
# plt.ylabel("Correlation: plan/problem sim")
# # plt.title("Problem Sim: Tok match (16.9%)")
# # plt.title("Problem Sim: UnivEnc (23.7%)")
# plt.title("Problem Sim: Gecko (29.2%)")
# # plt.title("Problem Sim: ST5 (25.2%)")
# # plt.title("Problem Sim: ST5_Large (24.8%)")
# # plt.title("Problem Sim: ST5_3b (23.7%)")

# # plt.title("Problem Sim: Gecko (no object) (23.9%)")
# # plt.title("Problem Sim: Gecko (with object) (27.3%)")
# # plt.title("Problem Sim: Gecko (with rel object) (20.7%)")

# plt.show()


# for i, sim in enumerate(pair_sim_mapping_problem):
#   print(i,end=":")
#   max = 0
#   max_ind = 0
#   for j, item in enumerate(sim):
#     if item > max and j != i:
#       max = item
#       max_ind = j
  # print(max_ind, max)


# high_problem_sim_pairs = """0:98 0.999079
# 1:156 0.99843425
# 2:187 0.9992458
# 3:50 0.9985832
# 4:186 0.9986255
# 5:11 0.9987471
# 6:89 0.99890363
# 7:168 0.99861884
# 8:7 0.997725"""

# readable_plan_print(data_lines[4])
# readable_plan_print_compare(data_lines[4], data_lines[186])
# readable_plan_print_compare(data_lines[4], data_lines[8])

In [ ]:
# problem_size_annotation = []

# # note that
# for i in range(data_scope):
#   problem_size_annotation.append(plans_df["inputs"][i].split("\n")[0].replace(")","").split("-")[-1])

# # print(len(problem_size_annotation))
# # print(problem_size_annotation)
# counter = defaultdict(int)
# for i,letter in enumerate(problem_size_annotation):
#   # counter[letter].append(all_correlation[i])
#   counter[letter] += 1

# # x, y = [],[]
# # for k,v in counter.items():
# #   x.append(int(k))
# #   # y.append(sum(v)/len(v))
# #   y.append(max(v))

# # index_order = list(np.argsort(y)[::-1])
# # new_x = [x[i] for i in index_order]
# # new_y = [y[i] for i in index_order]
# # fig = plt.figure(figsize = (10, 5))
# # plt.bar(new_x, new_y, color ='tab:blue', width = 0.4)
# # plt.xlabel("Problem Size")
# # plt.ylabel("Max correlation")

# int_keys = [int(x) for x in counter.keys()]
# size_ref = list(np.sort(int_keys))
# size_ref = [str(x) for x in size_ref]
# print(size_ref)


# list_of_lists = [[[] for x in range(len(size_ref))] for x in range(len(size_ref))]
# count_ref = [[0 for x in range(len(size_ref))] for x in range(len(size_ref))]

# for a, size_first in enumerate(size_ref):
#   for b, size_sec in enumerate(size_ref):
#     sel_problem_ind_first = []
#     sel_problem_ind_sec = []
#     for ind, size in enumerate(problem_size_annotation):
#       if size == size_first:
#         sel_problem_ind_first.append(ind)
#       if size == size_sec:
#         sel_problem_ind_sec.append(ind)

#     select_input_sim = []
#     select_plan_sim = []
#     for i in sel_problem_ind_first:
#       this_input_sim = []
#       this_plan_sim = []
#       for j in sel_problem_ind_sec:
#         this_input_sim.append(pair_sim_mapping_problem[i][j])
#         this_plan_sim.append(pair_sim_mapping[i][j])

#       select_input_sim.append(this_input_sim)
#       select_plan_sim.append(this_plan_sim)

#     all_correlation = []
#     for i, plan_sim in enumerate(select_input_sim):
#       input_sim = select_plan_sim[i]
#       res = stats.pearsonr(plan_sim, input_sim)[0]
#       list_of_lists[a][b].append(res)

# # print(list_of_lists)

# for i, this_list in enumerate(list_of_lists):
#   for j, sim in enumerate(this_list):
#     if args.heatmap == "mean":
#       # average similarity
#       list_of_lists[i][j] = round(sum(list_of_lists[i][j]) / len(list_of_lists[i][j]),2)
#     elif args.heatmap == "max":
#       list_of_lists[i][j] = max(list_of_lists[i][j])


# df = pd.DataFrame()
# df["sizes"] = size_ref

# for i, p1_size in enumerate(size_ref):
#     df[p1_size] = list_of_lists[i]

# df.set_index('sizes', drop=True, inplace=True)

# mask = np.triu(np.ones_like(df, dtype=bool))
# f, ax = plt.subplots(figsize=(9.5, 9))

# for i in range(len(size_ref)):
#   mask[i][i] = False

# cmap = sns.diverging_palette(230, 20, as_cmap=True)

# sns.heatmap(df,annot=True, cmap='gist_gray_r',cbar=False,vmax=1, center=0,
#     square=False, linewidths=.6, cbar_kws={"shrink": .5}, annot_kws={"fontsize":10})

# ax.set(xlabel="problem size from")
# ax.set(ylabel="problem size to")
# ax.tick_params(axis='x', labelsize= 15)
# ax.tick_params(axis='y', labelsize= 15)

In [ ]:
# Use an MLP to sanity check if we can improve the correlation
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

# embs = convert_tm_embedding(data_lines[:args.data_scope], em, args)

# ood_plans = pd.read_json(os.path.join(DATA_DIR, "planning/train_file_ood_8_20_v6.json"), lines=True, orient="records")
# ood_data_lines = ood_plans.to_dict('records')

# X_train, X_test, y_train, y_test = train_test_split(X, y,
#                                                     random_state=1)
# regr = MLPRegressor(random_state=1, max_iter=500).fit(X_train, y_train)

# Planning BPE

In [ ]:
# this is a compression problem with trade-off between the known stats and model capability
# one-way to learn this is using BPE to set what is a "token" unit to the model,
#   additionally, token freq can be used to fine-control the granuarity
# ideally, with high-level tokens, the models are easier to learn high-level features


# method 1 and 2: learning planning BPE from character/action-level with planning corpra
# part of the implementation is from https://www.geeksforgeeks.org/byte-pair-encoding-bpe-in-nlp/
import re
from collections import defaultdict

def get_stats(vocab, mode="char"):
    pairs = defaultdict(int)

    for word, freq in vocab.items():
      symbols = []
      if mode == "char":
        symbols = word.split()
      elif mode == "action":
        symbols = word.split("<a>")
      else:
        print("Unknown mode: %s" % mode)

      for i in range(len(symbols)-1):
          pairs[symbols[i],symbols[i+1]] += freq
    return pairs

def merge_vocab(pair, vocab, mode="char"):
    new_vocab = {}
    bigram, replacement = '', ''

    if mode == "char":
      bigram = ' '.join(pair)
      replacement = ''.join(pair)
    elif mode == "action":
      bigram = '<a>'.join(pair)
      replacement = ' '.join(pair)
    else:
      print("Unknown mode: %s" % mode)

    for word in vocab:
        new_word = word.replace(bigram, replacement)
        new_vocab[new_word] = vocab[word]
    return new_vocab

def get_vocab(data, mode="char"):
    vocab = defaultdict(int)
    for line in data:
      if mode == "char":
        for word in line.split():
            vocab[' '.join(list(word))] += 1
      elif mode == "action":
        vocab[line] += 1

    return vocab


def readable_plan_bpe(action_token):
  print(action_token.replace('<a>', ' '))

def byte_pair_encoding(data, args):
    vocab = get_vocab(data, mode=args.mode)
    for i in trange(args.iter):
        pairs = get_stats(vocab, mode=args.mode)
        try:
          best = max(pairs, key=pairs.get)
        except:
          print("Iter converges at %d" % i)
          break

        vocab = merge_vocab(best, vocab, mode=args.mode)
        # if i %10 == 0:
        #   print(vocab)
    return vocab

def bpe_presentation_main(plans_df, args):
  # print("Parsing plans...")
  parsed_plans = plan_parse(list(plans_df['targets']))
  action_lists = []

  for plan in parsed_plans:
    action_lists.extend(action_parse(plan, args))

  data = ["<a>".join(action_list) for action_list in action_lists]

  print("Char-level BPE")
  bpe_pairs_char = byte_pair_encoding(data, args)

  print("Char-level BPE:\n","Obj Centric",str(args.object_centric_parse),"|","Obj Affordance",str(args.object_centric_affordance),"|","Obj Relevance",str(args.object_relevance_parse))
  for k,v in bpe_pairs_char.items():
    if v > args.print_freq_cut:
      print("Freq %d" % v, end=": ")
      # print(k)
      readable_plan_bpe(k)

  return bpe_pairs_char



data_scope = 8700

bpe_args = SudoArgs()
bpe_args.data_scope = data_scope
bpe_args.print_detail = False # for debugging and visualization

bpe_args.consecutive = True # if not consecutive, we include the sublist
bpe_args.undir_metric = True # if the metric is undirection so that we can accelerate the compute
bpe_args.lca_with_reoccur = False # if we count re-occurence of the longest common actions

bpe_args.object_centric_parse = False # parse the actions by objects
bpe_args.object_centric_affordance = False # when parsing the actions, assume different locations with objects
bpe_args.object_relevance_parse = False # parse the actions by objects, add actions from relevant objects

bpe_args.score_conv = "mean-max" # max-max denotes  max(max(per object action series similarity))

# for BPE
bpe_args.iter = 500
bpe_args.mode = "action" # merge from char level or action level
bpe_args.print_freq_cut = 200

bpe_pairs = bpe_presentation_main(plans_df, bpe_args)

# Finding 0: there are some commonly shared patterns that might be useful
# e.g., unstack put-down unstack put-down pick-up stack
# Limitation: 1. not including the object (co-ref issue) or states
#             2. just co-occurence, no precedence

Char-level BPE


  0%|          | 0/500 [00:00<?, ?it/s]

Char-level BPE:
 Obj Centric False | Obj Affordance False | Obj Relevance False
Freq 1065: move
Freq 928: move move
Freq 402: move move move


In [ ]:
bpe_seqs = []

for k,v in bpe_pairs.items():
  if v >= 10:
    bpe_seqs.append(k)

print(len(bpe_seqs))

# compute the pair sim mapping for each bpe seq

258


In [ ]:
em = tm.models.GeckoEmbedder()
bpe_embs = tm.embedding(bpe_seqs, embedding_model_override=em)

In [ ]:
def find_pair_sim_mapping_bpe(plans_df, bpe_seqs, args):
  pair_sim_mapping = []
  pair_sim_interval = []

  parsed_plans = plan_parse(list(plans_df['targets']))

  action_lists = [action_parse(parsed_plan,args) for parsed_plan in parsed_plans]
  test_action_lists = [[seq.split(" ")] for seq in bpe_seqs]


  # design choice: sampling 100 data points, 30.72 it/s required for each pair
  for i, action_list0 in enumerate(tqdm(test_action_lists)):
    this_pair_sim_mapping = []
    this_pair_sim_interval = []

    for j, action_list1 in enumerate(action_lists[:args.data_scope]):
      # if we assume a undirectioned metric
      if args.undir_metric and j < i:
        this_pair_sim_mapping.append(pair_sim_mapping[j][i])
        continue

      common_list_collection,common_len_collection = [],[]
      common_list = [] # dummy
      common_a1_interval = [0,0]
      max_common_len = 0

      for a0 in action_list0:
        this_common_list, this_common_len = [],[]
        for a1 in action_list1:
          common_list, common_a1_interval = longest_common_actions(a0, a1, args)
          common_len = len(common_list)
          # consider the case where there are multiple common_list
          if args.lca_with_reoccur:
            count0 = " ".join(a0).count(" ".join(common_list))
            count1 = " ".join(a1).count(" ".join(common_list))
            factor = max(count0, count1)
            common_len = factor * common_len

          # design choice: how and if we should penalize the length
          common_len = common_len**2 / (len(a0) * len(a1))
          if common_len > max_common_len:
            max_common_len = common_len

          this_common_list.append(common_list)
          this_common_len.append(common_len)

        common_list_collection.append(this_common_list)
        common_len_collection.append(this_common_len)

      sim_score = 0
      if args.score_conv == "mean-max":
        temp_score = [max(x) for x in common_len_collection]
        sim_score = sum(temp_score) / len(temp_score)

      if args.score_conv == "max-max":
        sim_score = max_common_len

      this_pair_sim_mapping.append(sim_score)
      this_pair_sim_interval.append(common_a1_interval)

    pair_sim_mapping.append(this_pair_sim_mapping)
    pair_sim_interval.append(this_pair_sim_interval)

  return pair_sim_mapping, pair_sim_interval

pair_sim_mapping_bpe, pair_sim_interval_bpe = find_pair_sim_mapping_bpe(plans_df, bpe_seqs, args)

  0%|          | 0/258 [00:00<?, ?it/s]

In [ ]:
# compute the signal
with gfile.Open('/cns/od-d/home/xinranzhao/pair-sim-maps/block_dev_v6_inputs_embs.json', mode='r') as f:
  test_inputs_embs = json.load(f)

with gfile.Open('/cns/od-d/home/xinranzhao/pair-sim-maps/block_dev_v6_targets_embs.json', mode='r') as f:
  test_targets_embs = json.load(f)

# # with gfile.Open('/cns/od-d/home/xinranzhao/pair-sim-maps/pair_sim_mapping_bpe.json', mode='w') as f:
# #   json.dump(pair_sim_mapping_bpe, f)

# with gfile.Open('/cns/od-d/home/xinranzhao/pair-sim-maps/pair_sim_mapping_bpe.json', mode='r') as f:
#   pair_sim_mapping_bpe = json.load(f)

In [ ]:
pair_sim_mapping_bpe_merged = []
parsed_plans = plan_parse(list(plans_df['targets']))
action_lists = [action_parse(parsed_plan,args) for parsed_plan in parsed_plans]
test_action_lists = [[seq.split(" ")] for seq in bpe_seqs]
# print(action_lists[0])
# print(test_action_lists[0])
# print(" ".join(test_action_lists[0][0]) in " ".join(action_lists[0][0]))

for i in trange(args.test_scope):
  temp = []

  for j in range(len(bpe_embs)):
    sim = cosine_similarity([test_inputs_embs[i]], [bpe_embs[j]])[0][0]
    temp.append(sim)

  # for j in range(len(bpe_seqs)):
  #     a0 = test_action_lists[j][0]
  #     a1 = action_lists[i][0]
  #     common_list, common_a1_interval = longest_common_actions(a0, a1, args)
  #     common_len = len(common_list)
  #     temp.append(common_len**2 / (len(a0) * len(a1)))

  # reweight by bpe seq length?
  mean = statistics.mean(temp)
  std = statistics.stdev(temp)

  selected_token_index, score_coeff = [],[]

  for j, score in enumerate(temp):
    if score == max(temp) or score > (mean+1*std):
      selected_token_index.append(j)
      score_coeff.append(score)

  # print(len(selected_token_index))

  item_scores = [0 for x in range(args.data_scope)]
  for j, token_index in enumerate(selected_token_index):
    raw_scores = pair_sim_mapping_bpe[token_index]
    for k, score in enumerate(raw_scores[:args.data_scope]):
      item_scores[k] += (score_coeff[j])*score

  pair_sim_mapping_bpe_merged.append(item_scores)

  0%|          | 0/300 [00:00<?, ?it/s]

In [ ]:
print(len(pair_sim_mapping_bpe_merged[0]))
print(pair_sim_mapping_bpe_merged[1])

28000
[1.882238936255213, 2.4413292472649255, 1.3582557890477842, 1.86812503075445, 1.6427769820628768, 2.0305721106039973, 1.3130442108924385, 1.5767521518019523, 1.8139940299731379, 2.177603017383449, 2.0553073570155553, 2.136589820256639, 1.6905730524590032, 2.4283262146120186, 1.9444139476391467, 2.535859578688505, 1.222668488404365, 2.460797641964364, 2.2798702705080016, 2.744343170150151, 0.8452865262295016, 2.7149495539274913, 1.6244576884831978, 2.77573152782142, 2.8292585282197535, 1.86812503075445, 1.86812503075445, 2.535859578688505, 1.5767521518019523, 2.695554824520717, 2.576455826610238, 1.3582557890477842, 2.4166186351430228, 1.6905730524590032, 2.982428789944529, 1.86812503075445, 2.177603017383449, 1.8487974837084056, 2.461829941060122, 1.9444139476391467, 2.0305721106039973, 0.8452865262295016, 4.174164893418673, 1.86812503075445, 1.9444139476391467, 1.7810299199739617, 1.0222891997959234, 1.7054041286502202, 2.2798702705080016, 2.4166186351430228, 1.788718984362506, 

## BPE for other domains

In [ ]:
# BPE with other problems
# Finding 3: OOD and ID has shared long seq BPE, e.g., unstack_1 unstack_0 stack_0 stack_1 unstack_1 unstack_0 stack_1 unstack_1 stack_0 stack_1



# block_file = os.path.join(DATA_DIR, "planning/train_file_pddl_plan_3_20_v6.json")
# ood_file = os.path.join(DATA_DIR, "planning/train_file_ood_8_20_v6.json")
# grid_file = os.path.join(DATA_DIR, "planning/training_grid_v2.json")
# logistics_file = os.path.join(DATA_DIR, "planning/training_logistics_v5.json")
# tetris_file = os.path.join(DATA_DIR, "planning/training_tetris_v1.json")
# trip_plan_jsonl = os.path.join(DATA_DIR, "planning/trip_planning.jsonl")

# file_map = {
#     "block": block_file,
#     "ood": ood_file,
#     "grid": grid_file,
#     "logistics": logistics_file,
#     "tetris": tetris_file,
# }

# def file_wise_bpe_main(file_name, args):
#   print(file_name)
#   plans_df = pd.read_json(file_map[file_name], lines=True, orient="records")
#   bpe_presentation_main(plans_df, args)

# args.print_freq_cut = 1000

# # ood:
# file_wise_bpe_main("ood", args)

# filename = "logistics"
# readable_index_map = {"grid":17, "logistics":18, "tetris":17}
# plans_df = pd.read_json(file_map[filename], lines=True, orient="records")
# data_lines = plans_df.to_dict('records')
# data_line_item = data_lines[readable_index_map[filename]]
# if filename == "logistics":
#   args.print_freq_cut = 5000
# readable_plan_print(data_line_item,option="input")
# bpe_presentation_main(plans_df, args)

# explore BPE with NL problem discription
# start with jsonl
# tools of keyword

# trip_plan_jsonl = os.path.join(DATA_DIR, "trip_planning.jsonl")

# plans_df = pd.read_json(trip_plan_jsonl, lines=True, orient="records")
# data_lines = plans_df.to_dict('records')
# print(data_lines[0].keys())

# print(data_lines[0]["task"])
# print(data_lines[0]["solution"])

# Exemplar pipeline with Gemini

## Usage of the Evergreen API

In [ ]:
# planning prompt:

# test my ideas on auto-annotation
prompt_example = """
Please solve the problem:
(define (problem BW-rand-4)
(:domain blocksworld-4ops)
(:objects b4 b1 b3 b2)
(:init
(on b3 b1)
(on b1 b4)
(clear b3)
(handempty)
(ontable b2)
(ontable b4)
(clear b2)
)
(:goal (and
(on b2 b4)
(on b3 b1)
))
)
Your plan as plain text without formatting:
(unstack b3 b1)
(put-down b3)
(unstack b1 b4)
(put-down b1)
(pick-up b2)
(stack b2 b4)
(pick-up b3)
(stack b3 b1)
done.
"""
input1 = """
(define (problem BW-rand-7)
(:domain blocksworld-4ops)
(:objects b4 b3 b5 b2 b7 b6 b1)
(:init
(on b1 b6)
(clear b7)
(on b2 b5)
(ontable b6)
(clear b3)
(ontable b5)
(ontable b4)
(handempty)
(on b3 b4)
(clear b1)
(on b7 b2)
)
(:goal (and
(on b7 b1)
(on b6 b7)
(on b4 b6)
))
)
"""

input2 = """Please solve the problem:
(define (problem BW-rand-6)
(:domain blocksworld-4ops)
(:objects b1 b5 b2 b4 b3 b6)
(:init
(on b2 b5)
(clear b3)
(on b4 b2)
(handempty)
(on b3 b4)
(ontable b1)
(on b5 b1)
(ontable b6)
(clear b6)
)
(:goal (and
(on b5 b4)
(on b3 b5)
))
)
Your plan as plain text without formatting:
"""


# m.query(prompt_example+input1,max_output_tokens=200)


In [ ]:
# test the planning performance on OOD dataset with:
#       no retrieval,
#       problem-sim retrieval,
#       plan-sim retrieval,
#       plan-problem-mix-sim retrieval
#       Gecko retrieval
# expand the performance with
# metric: matched action


# Usable: GEMINI_M_IT, GEMINI_XXS_IT, GEMINI_M_IT (has rate limit), GEMINI_M_IT_BULK, GEMINI_XL_IT, GEMINI_GOLDFISH(1.5 pro), GEMINI_GOLDFISH_SHORT, GEMINI_S_FLASH
# model_url = model_client.ModelEntry.GEMINI_M_IT # @param {isTemplate: true}
# check this page for more info: https://g3doc.corp.google.com/learning/deepmind/evergreen/g3doc/shared_servers.md?cl=head#fair-use-policy

# model_url = model_client.ModelEntry.GEMINI_GOLDFISH

# client = model_client.ModelClient(
#     model_url=model_url,
# )

# client.generate("Generate an image for:"+input1)

# tokenize
# Tokenization is only supported for Evergreen models.
# response = client.tokenize(
#     'If you could describe chocolate in 5 words, what would they be?'
# )

# # Tokens can be accessed from the `tokens` field.
# for _, chunk in response.items():
#   print(chunk.tokens)


# def print_content_with_tokens(content: content_api.Content) -> None:
#   token_ids = []
#   logprobs = []
#   for _, chunk in content.items():
#     token_ids.extend(chunk.tokens.token_ids)
#     logprobs.extend(chunk.tokens.logprob)

#   print(content_api.as_text(content))
#   print(f'{token_ids=}')
#   print(f'{logprobs=}')

# log probs
# response = client.log_likelihood(
#     'If you could describe chocolate in 5 words, what would they be?',
#     ['Sweet', 'delectable', 'Ineffable']
# )
# print_content_with_tokens(response.prompt)
# for completion in response:
#   print(f'{completion.log_likelihood=}')
#   print_content_with_tokens(completion.completion)

# # readable_plan_print_compare(data_lines[4], data_lines[186])
# # readable_plan_print_compare(data_lines[4], data_lines[8])

# problem_sim_prompt = prompt_template.replace('<input>', data_lines[8]["inputs"]).replace('<plan>', data_lines[8]["targets"])

# response = client.generate(problem_sim_prompt+input2) #
# print(model_client.as_text(response), end='')

## ICL Pipeline


In [ ]:
# ad_hoc eval with LCA

In [170]:
# start in-context learning
# VAL will be run separately on the cloudtop

data_scope = 2400
test_scope = 300

args = SudoArgs()
args.data_scope = data_scope
args.test_scope = test_scope
args.print_detail = False # for debugging and visualization

args.consecutive = True # if not consecutive, we include the sublist
args.undir_metric = False # if the metric is undirection so that we can accelerate the compute
args.lca_with_reoccur = False # if we count re-occurence of the longest common actions

args.exec_based = False # use the execution of the plans, instead of the pure plans
args.init_exec = False # use the input as the initial condition of the plans

args.partial_plan_gen = False # convert the demos into partial problems generated by the partial plans
args.goal_infer_with_on = False # if we ensure the generated plan goals are with "on"

args.object_centric_parse = False # parse the actions by objects
args.object_centric_affordance = False # when parsing the actions, assume different locations with objects
args.object_relevance_parse = False # parse the actions by objects, add actions from relevant objects

args.score_conv = "mean-max" # max-max denotes  max(max(per object action series similarity))
args.score_conv_mode = "weighted" # if weighted, the overall common_len will be weighed by the common len of each
args.heatmap = "mean" # type of heatmap

args.use_prompt = False # use prompt during embedding
args.plan_sim = "token" # token or embedding model names: [gecko,retsim,uni,unil,st5,st5l,st53b]
args.client = "goldfish" # m, xxs_it, m_it, m_it_bulk, xl_it, goldfish, goldfish_s, s_flash
args.num_demo = 1 # test: 0,1,4,8

args.icl_sim = "asr" # [random, plan, input, mlp, bpe, asr] similarity to sample the icl examples
args.icl_sim_mode = "static" # [static, dynamic, reverse, cluster, dcluster] if dynamic, we sample demostration with distribution, num_demo become the std offset. dcluster is a dynamic number of demo
args.cluster_size = 8 # how many examples per cluster to search
args.per_cluster_sample_max = 2 # how examples per cluster to smaple

args.debug_mode = False

MAX_RETRIES = 80  # Maximum number of retries
RETRY_INTERVAL = 10  # Seconds between retries

embedder_map = {
    "token":None,
    # "retsim":tm.models.RetsimEmbedder(),
    "gecko":tm.models.GeckoEmbedder(),
    # "uni":tm.models.UnivSentEnc(),
    # "unil":tm.models.UnivSentEncLarge(),
    # "st5":tm.models.ST5Embedder(),
    # "st5l":tm.models.ST5Embedder_large(),
    # "st53b":tm.models.ST5Embedder_3b(),
}


# Usable: GEMINI_M, GEMINI_XXS_IT, GEMINI_M_IT (has rate limit), GEMINI_M_IT_BULK, GEMINI_XL_IT, GEMINI_GOLDFISH(1.5 pro), GEMINI_GOLDFISH_SHORT, GEMINI_S_FLASH
# model_url = model_client.ModelEntry.GEMINI_M_IT # @param {isTemplate: true}
# check this page for more info: https://g3doc.corp.google.com/learning/deepmind/evergreen/g3doc/shared_servers.md?cl=head#fair-use-policy

temperature = 0.0
max_tokens = 1600
seed = 42

sampling_config = model_client.SamplingConfig.TopK(
    temperature=temperature
)

# Create a config with max num tokens to generate and sampling_config.
config = model_client.GenerationConfig(
    seed=seed,
    token_generation=model_client.make_token_generation_config(
        sampling_config=sampling_config,
        length=max_tokens,
        early_stopping_suffixes=None,
    ),
    write_cache=False,
    read_cache=False,
)


model_url_map = {
    "m": model_client.ModelEntry.GEMINI_M,
    "xxs_it": model_client.ModelEntry.GEMINI_XXS_IT,
    "m_it": model_client.ModelEntry.GEMINI_M_IT,
    "m_it_bulk": model_client.ModelEntry.GEMINI_M_IT_BULK,
    "xl_it": model_client.ModelEntry.GEMINI_XL_IT,
    "goldfish": model_client.ModelEntry.GEMINI_GOLDFISH,
    "goldfish_s": model_client.ModelEntry.GEMINI_GOLDFISH_SHORT,
    "s_flash": model_client.ModelEntry.GEMINI_S_FLASH,
}

em = embedder_map[args.plan_sim]

# print(str(model_url_map[args.client].value))

client = model_client.ModelClient(
    # model_url= model_url_map[args.client],
    model_url="evergreen2://blade:gdm-aip-agent-generate-service-prod/lmroot:goldfish_shared",
    route = model_client.Route.LM_PROXY,
    # Enables automatic retries, ideal for
    # batch traffic, see go/lm-proxy-quota for more info.
    stub_options = model_client.BULK_STUB_OPTIONS,
    default_config=config,
)

# client = model_client.ModelClient(
#     model_url= "evergreen://blade:gdm-aip-agent-generate-service-prod/lmroot:blade:aip-serving-bohnetbd-gemini_mpp_1m",
#     route = model_client.Route.LM_PROXY,
#     # Enables automatic retries, ideal for
#     # batch traffic, see go/lm-proxy-quota for more info.
#     stub_options = model_client.BULK_STUB_OPTIONS,
# )

response = client.generate("It is a lovely day!")
print(model_client.as_text(response))

I'm glad to hear it! I hope you're enjoying it.  What makes it so lovely?



## In-house Model Server

In [ ]:
# # loading external models
# # @title Select and connect to model

# # blaze run //learning/gemini/human_evals/colabs:eval_colab
# # https://g3doc.corp.google.com/learning/gemini/cms/g3doc/evals/third_party_apis.md?cl=head



# model_name_str = 'mpp_rev11' # @param ['gpt-4o-2024-05-13','sv2_rev11', 'mpp_rev11', 'mpp_rev11nq', 'sv2_rev10', 'gemini_lpp-rlhf-rev1-or-mm', 'gemini_1.5', 'gemini_1.5_quant','gemini_xl', 'gemini_lpp_sft', 'gpt4', 'gpt4_turbo_20240409', 'claude-3', 'mpp_rev8', 'mpp_rev10d']
# model_name = model_name_str
# model_name_for_filename = model_name_str



# if 'gpt' in model_name_str or 'claude' in model_name_str:
#   from google3.learning.deepmind.boq.data.ditto.client import ditto
#   from google3.learning.gemini.human_evals.sampling import anthropic
#   from google3.learning.gemini.human_evals.sampling import openai
#   from google3.learning.gemini.human_evals.sampling import sampler

# GPT_API_KEY= "gemini_evals_openai_2"
# if model_name == 'gpt4':
#   GPT_MODEL = "gpt-4-turbo-preview'"
#   print(GPT_MODEL)
# elif model_name == 'gpt4_turbo_20240409':
#   GPT_MODEL = "gpt-4-turbo-2024-04-09"
#   print(GPT_MODEL)
# elif model_name == 'gpt-4o-2024-05-13':
#   GPT_MODEL = "gpt-4o-2024-05-13"
#   print(GPT_MODEL)
# else:
#   GPT_MODEL = None
# import time
# import tensorflow as tf
# import tensorflow.io.gfile as gfile
# import csv


# from google3.learning.deepmind.evergreen.model_access.client.python import model_client
# from google3.learning.deepmind.evergreen.model_access.client.python import content_api

# from google3.third_party.sentencepiece.src.python import sentencepiece_processor as spm_py


# class RetryError(Exception):
#   """Custom exception class for retry failures."""
#   pass


# class Generator(object):
#   """Provides access to diverse models in a uniform way."""

#   def __init__(
#       self,
#       model_str: str,
#       model_path: str,
#       retry: int = 20,
#       max_tokens: int = 1536,
#       temperature: float = 0.0,
#       seed:int=101,
#       wrap_fun=None):
#     """Initializer.

#     Args:
#       model_str: model name either ulm340 or gemini_xl.
#       model_path: with ulm340b, e.g. the sax server. with gemini_xl, e.g. the
#         gemini_xl_it server.
#       retry: Number of retries to query server.
#       max_tokens: Number of tokens to generate.
#       temperature: The used temperature when generating.

#     """
#     self.model_str = model_str.lower()
#     self.spm = spm_py.SentencePieceProcessor()
#     self.retry = retry
#     self.seed = seed
#     self.wrap_fun = wrap_fun
#     self.max_tokens = max_tokens
#     self.temperature = temperature
#     print(self.model_str)
#     if model_str == 'claude-3':
#       self.ditto_client = ditto.DittoClient(ditto_config_id='gemini_evals_anthropic')
#       self.anthropic_client = anthropic.AnthropicClient(self.ditto_client)
#     elif 'gpt4' in model_str:
#       pass
#     elif 'gpt-4o' in model_str:
#       pass
#     else:

#       # connect to evergreen server gemini
#       print(f'init {model_str} {model_path}')

#       # set the temperature
#       sampling_config = model_client.SamplingConfig.TopK(
#           temperature=temperature
#       )

#       # Create a config with max num tokens to generate and sampling_config.
#       config = model_client.GenerationConfig(
#           seed=self.seed,
#           token_generation=model_client.make_token_generation_config(
#               sampling_config=sampling_config,
#               length=max_tokens,
#               early_stopping_suffixes=None,
#           ),
#           write_cache=False,
#           read_cache=False,
#       )

#       # Finally create the client
#       self.client = model_client.ModelClient(
#           model_url=model_path,
#           default_config=config,
#       )

#     self.spm.Load(
#         '/placer/prod/home/gemini-data-access/shareable_checkpoints/tokenizers/v1/gemini_tier4/256000_with_task_markers_start_of_image.model'
#     )


#   def anthropic_request(self, prompt: str) -> str:
#     """Here we go for Anthropic."""
#     print('claude model:', CLAUDE_MODEL)
#     sampling_options = anthropic.sampler.SamplingOptions(
#         max_tokens_to_sample=self.max_tokens,
#         temperature = 0.0
#     )

#     responce = self.anthropic_client.messages(
#             messages=[
#                 anthropic.Message(
#                     role=anthropic.Role.USER, content=prompt
#                 )
#             ],
#             model = CLAUDE_MODEL ,
#             sampling_options = sampling_options,
#         )

#     return responce['content'][0]['text']

#   def request_for_chat_completion_api(self, prompt: str) -> str:
#     """Here we go for gpt4."""
#     print('gpt4', GPT_MODEL)
#     ditto_client = ditto.DittoClient(
#         ditto_config_id=GPT_API_KEY)
#     openai_client = openai.OpenAIClient(ditto_client)
#     sampling_options = sampler.SamplingOptions(
#         max_tokens_to_sample=self.max_tokens,
#         temperature=self.temperature,
#     )
#     messages = [
#         openai.SystemMessage(
#             content="You are a helpful assistant."
#         ),
#         openai.UserMessage(content=prompt),
#     ]
#     response = openai_client.complete(
#         model=GPT_MODEL,
#         messages=messages,
#          sampling_options=sampling_options,
#     )
#     return response["choices"][0]["message"]["content"]



#   def generate(self, text: str):
#     """Use a LLM to generate a text.

#     Args:
#       text: input text.

#     Raises:
#       RetryError: in cases too many retries.

#     Returns:
#       generated text.
#     """
#     if self.model_str == 'claude-3':
#       print('claude model:', self.model_str)
#       return self.anthropic_request(text)
#     elif 'gpt' in self.model_str:
#       return self.request_for_chat_completion_api(text)
#     else:
#       # print('some google model:', self.model_str)
#       retry = self.retry
#       while retry > 0:
#         try:
#           response = self.client.generate(text)
#           text = model_client.as_text(response)
#           text = text.replace("<ctrl100>", "")

#           return text
#         except Exception as e:  # pylint: disable=broad-exception-caught
#           print(f'I will retry {retry}. Error while generating: {str(e)}')
#           retry -= 1
#           time.sleep(3)
#           # raise RetryError('Retry to query server did not work.')

#   def spm_length(self, text: str):
#     return len(self.spm.EncodeAsIds(text))



# if model_name_str == 'gemini_1.5_quant':
#   model_url = 'evergreen:///mbns/vz/home/courier/bohnetbd/g_bb_vlp'
#   model_name_for_filename = 'gemini_1.5q'
# elif model_name_str == 'gemini_1.5':
#   model_name_for_filename = 'gemini_1.5'
#   model_url = 'evergreen:///mbns/iz/home/courier/kswersky/gf_bb1_vf_2'
# elif model_name_str == 'gemini_lpp_sft':
#   model_name_for_filename = 'gemini_lpp_sft'
#   model_url = 'evergreen:///mbns/vz/home/courier/bohnetbd/lpp_vf_1m_disagg_test'
# elif model_name_str == 'gemini_lpp-rlhf-rev1-or-mm':
#   model_name_for_filename = 'lpp-rlhf-rev1-or-mm'
#   model_url = 'evergreen:///mbns/iz/home/courier/tianheyu/lpp-rlhf-rev1-or-mm'
# elif model_name_str == 'gpt4':
#   model_name_for_filename = 'gpt4'
#   model_url = None
# elif model_name_str == 'gpt4_turbo_20240409':
#   model_name_for_filename = 'gpt4_turbo_20240409'
#   model_url = None
# elif model_name_str == 'gpt-4o-2024-05-13':
#   model_name_for_filename = 'gpt-4o-2024-05-13b'
#   model_url = None
# elif model_name_str == 'claude-3':
#   model_name_for_filename = 'claude-3'
#   model_url = None
# elif model_name_str == 'mpp_rev8':
#   model_url = 'evergreen://blade:gdm-aip-agent-generate-service-prod/lmroot:goldfish_shared'
# elif model_name_str == 'mpp_rev10d':
#   # model_url = 'evergreen://blade:gdm-aip-agent-generate-service-dev/lmroot:mpp_evals' they wrote that it had a bug
#   model_url = 'evergreen://blade:aip-agent-evals-dedicated/lmroot:mpp_evals'
#   model_url = "evergreen://blade:aip-agent-evals-dedicated/lmroot:mpp_evals"

# elif model_name_str == 'mpp_rev11':
#   model_url = 'evergreen:///mbns/li/home/courier/biaojiaxing/mpp_rev11_ioreport'
#   model_url = "evergreen://blade:gdm-aip-agent-generate-service-prod/lmroot:blade:aip-serving-bb_rev11_1m_b1-gemini_mpp_or_rev11_1m_b1"
#   model_url = "evergreen://blade:gdm-aip-agent-generate-service-prod/lmroot:blade:aip-serving-bb_rev11_1m_b1-gemini_mpp_or_rev11_1m_b1"
#   model_url = "evergreen://blade:gdm-aip-agent-generate-service-prod/lmroot:blade:aip-serving-shared-gemini_mpp_or_rev11_32k_b1"

#   model_url = "evergreen://blade:gdm-aip-agent-generate-service-prod/lmroot:blade:aip-serving-bb_rev11_32k_b1-gemini_mpp_or_rev11_32k_b1"
#   model_url = "evergreen:///mbns/iz/home/courier/bb_rev11_mpp_128k_b1/bb_rev11_mpp_128k"
#   model_url = "evergreen://blade:gdm-aip-agent-generate-service-prod/lmroot:blade:aip-serving-bb_rev11_1m_b1-gemini_mpp_or_32k_rev11_1m_b2"
#   model_url = "evergreen://blade:gdm-aip-agent-generate-service-prod/lmroot:goldfish_shared"
# elif model_name_str == 'mpp_rev11nq':
#   model_url = "evergreen:///mbns/vz/home/courier/bohnetbd/bb_15_pro"

# elif model_name_str == 'sv2_rev10':

#   model_url = "evergreen://blade:gdm-aip-agent-generate-service-dev/lmroot:v2_s_dense_evals"
#   model_url = "evergreen://blade:gdm-aip-agent-generate-service-prod/lmroot:blade:aip-serving-mauger-gemini_s_v2_dense_rev11_2m_1"
# elif model_name_str == 'sv2_rev11':
#   model_url = "evergreen:///mbns/vz/home/courier/charlinel/xid_110107956_step_1800"
#   model_url = "evergreen:///mbns/li/home/courier/philhc/26b_rev11_2m_no_cache"
#   model_url = "evergreen://blade:gdm-aip-agent-generate-service-dev/lmroot:v2_s_dense_evals_rev11"
# else:
#   raise ValueError(f'Unknown model name: {model_name_str}')



# CLAUDE_MODEL = "claude-3-opus-20240229" # @param {'type': 'string'}
# # gemini for all Gemini
# generator = Generator(model_name_str,
#                       model_path=model_url,
#                       retry=500,
#                       max_tokens=1024,
#                       temperature=0.0)

## ICL pipeline with Evergreen

In [171]:
# step 1 create a store different kinds of exemplar mapping
# pair_sim_mapping, pair_sim_interval = find_pair_sim_mapping_main(plans_df, test_df, train_exec, test_exec, args)
# pair_sim_mapping, pair_sim_interval = find_pair_sim_mapping_main(plans_df, ood_df, train_exec, ood_exec, args) # ood plan_similarity

# pair_sim_mapping_input = find_input_sim_main(plans_df, test_df, em, args) # input similarity


# MLP based performance, using learned MLP to get the score
def mlp_sim_score(em, regr, plans_df, test_df, args):
  data_lines = plans_df.to_dict(orient="records")
  test_lines = test_df.to_dict(orient="records")

  data_inputs_embs = convert_tm_embedding(data_lines[:args.data_scope], em, args)
  test_inputs_embs = convert_tm_embedding(test_lines, em, args)
  data_targets_embs = convert_tm_embedding(data_lines[:args.data_scope], em, args, keyword="targets")

  # use pre-computed embedding to save time
  # with gfile.Open('/cns/od-d/home/xinranzhao/pair-sim-maps/block_train_v6_inputs_embs.json', mode='r') as f:
  #   data_inputs_embs = json.load(f)
  # with gfile.Open('/cns/od-d/home/xinranzhao/pair-sim-maps/block_dev_v6_inputs_embs.json', mode='r') as f:
  #   test_inputs_embs = json.load(f)
  # with gfile.Open('/cns/od-d/home/xinranzhao/pair-sim-maps/block_train_v6_targets_embs.json', mode='r') as f:
  #   data_targets_embs = json.load(f)

  mlp_sim_mapping = []

  for ind2 in trange(args.test_scope):
    X_test = []

    for ind1 in range(args.data_scope):
      temp = []
      temp.extend(data_inputs_embs[ind1])
      temp.extend(data_targets_embs[ind1])
      temp.extend(test_inputs_embs[ind2])
      X_test.append(temp)

    y_test = regr.predict(X_test)
    mlp_sim_mapping.append(y_test)

  return mlp_sim_mapping


In [172]:
def find_pair_sim_mapping_sq(plans_df, args):
  # for clustering based method
  pair_sim_mapping = []

  parsed_plans = plan_parse(list(plans_df['targets']))

  action_lists = [action_parse(parsed_plan,args) for parsed_plan in parsed_plans]

  for i, action_list0 in enumerate(action_lists):
    this_pair_sim_mapping = []

    for j, action_list1 in enumerate(action_lists):
      # if we assume a undirectioned metric
      if args.undir_metric and j < i:
        this_pair_sim_mapping.append(pair_sim_mapping[j][i])
        continue

      common_list_collection,common_len_collection = [],[]
      common_list = [] # dummy
      common_a1_interval = [0,0]
      max_common_len = 0

      for a0 in action_list0:
        this_common_list, this_common_len = [],[]
        for a1 in action_list1:
          common_list, common_a1_interval = longest_common_actions(a0, a1, args)
          common_len = len(common_list)
          # consider the case where there are multiple common_list
          if args.lca_with_reoccur:
            count0 = " ".join(a0).count(" ".join(common_list))
            count1 = " ".join(a1).count(" ".join(common_list))
            factor = max(count0, count1)
            common_len = factor * common_len

          # design choice: how and if we should penalize the length
          common_len = common_len**2 / (len(a0) * len(a1))
          if common_len > max_common_len:
            max_common_len = common_len

          this_common_list.append(common_list)
          this_common_len.append(common_len)

        common_list_collection.append(this_common_list)
        common_len_collection.append(this_common_len)

      # organize with mean(max(per object action series))
      sim_score = 0
      if args.score_conv == "mean-max":
        temp_score = [max(x) for x in common_len_collection]
        sim_score = sum(temp_score) / len(temp_score)

      if args.score_conv == "max-max":
        sim_score = max_common_len

      this_pair_sim_mapping.append(sim_score)

    pair_sim_mapping.append(this_pair_sim_mapping)

  return pair_sim_mapping

In [173]:
# filename = "dev-300/dev-300-28000-as-token-goldfish-10-random-static.json"

#filename = "dev-300/new-dev-300-28000-as-token-goldfish-4-asr-static.json"

# filename = "dev-300/iter1-dev-300-28000-as-token-goldfish-40-asr-static.json"


# filename = "dev-300/grid/dev-300-8700-as-token-goldfish-100-random-static.json"

# filename = "dev-300/logistics/dev-300-82000-as-token-goldfish-40-random-static.json"
# filename = "dev-300/logistics/dev-300-82000-as-token-goldfish-4-asr-static.json"

# filename = "dev-300/tetris/dev-300-2400-as-token-goldfish-40-random-static.json"

# filename = "dev-300/ood/dev-300-28000-as-token-goldfish-4-random-static.json"
filename = "dev-300/tetris/dev-300-2400-as-token-goldfish-100-asr-static.json"

with gfile.Open('/cns/od-d/home/xinranzhao/'+filename, 'r') as f:
  asr_output_collection = json.load(f)

In [174]:
# Prepare the pair-sim-mapping for Plan Similarity and Action Sequence Re-sampling

# print(pair_sim_mapping_mlp[0])
# filename = "pair-sim-mlp-trial-27000.json"
# # with gfile.Open('/cns/od-d/home/xinranzhao/pair-sim-maps/'+filename, 'w') as f:
# #   json.dump([x.tolist() for x in pair_sim_mapping_mlp], f)

# with gfile.Open('/cns/od-d/home/xinranzhao/pair-sim-maps/'+filename, 'r') as f:
#   pair_sim_mapping_mlp = json.load(f)

# pair_sim_mapping, pair_sim_interval = find_pair_sim_mapping_main(plans_df, test_df, train_exec, test_exec, args) # plan_similarity

# filename = "logistics-pair-sim-plan-token-300-28000.json"
# with gfile.Open('/cns/od-d/home/xinranzhao/pair-sim-maps/'+filename, 'r') as f:
#   pair_sim_mapping = json.load(f)

# print(list(test_df['targets'])[0])
# print("--")
# print(output_collection[0][1])
# ASR
asr_targets = []
for item in asr_output_collection:
  asr_targets.append(item[-1])

asr_test_df = copy.deepcopy(test_df) # test_df.copy()

for i in range(len(asr_test_df['targets'])):
  if i < len(asr_targets):
    continue
  else:
    asr_targets.append("out of scope")

asr_test_df['targets'] = asr_targets

pair_sim_mapping_asr, pair_sim_interval = find_pair_sim_mapping_main(plans_df, asr_test_df, train_exec, test_exec, args)

  0%|          | 0/300 [00:00<?, ?it/s]

In [ ]:

# filename = "logistics-pair-sim-mapping-asr-iter1.json"
# with gfile.Open('/cns/od-d/home/xinranzhao/pair-sim-maps/'+filename, 'w') as f:
#   json.dump(pair_sim_mapping_asr, f)

# # pair_sim_mapping_input= find_input_sim_main(plans_df, test_df, em, args) # input similarity

# # filename = "logistics-pair-sim-input-token-300-28000.json"
# # filename = "grid-pair-sim-input-gecko-300-8700.json"

# # with gfile.Open('/cns/od-d/home/xinranzhao/pair-sim-maps/'+filename, 'r') as f:
# #   pair_sim_mapping_input = json.load(f)

# print(len(pair_sim_mapping_asr[0]))

In [ ]:
# filename = "ood-pair-sim-plan-token-300-28000.json"

# # with gfile.Open('/cns/od-d/home/xinranzhao/pair-sim-maps/'+filename, 'w') as f:
# #   json.dump(pair_sim_mapping, f)

# with gfile.Open('/cns/od-d/home/xinranzhao/pair-sim-maps/'+filename, 'r') as f:
#   pair_sim_mapping = json.load(f)

# filename = "ood-pair-sim-input-token-300-28000.json"

# # with gfile.Open('/cns/od-d/home/xinranzhao/pair-sim-maps/'+filename, 'w') as f:
# #   json.dump(pair_sim_mapping_input, f)

# with gfile.Open('/cns/od-d/home/xinranzhao/pair-sim-maps/'+filename, 'r') as f:
#   pair_sim_mapping_input = json.load(f)

# pair_sim_mapping_mlp = mlp_sim_score(em, regr_reload, plans_df, test_df, args)

# filename = "pair-sim-mlp-esv-28000.json"
# filename = "pair-sim-mlp2-as-300-1000-28000.json"
# filename = "grid-pair-sim-mlp2-as-300-400-8700.json"
# filename = "tetris-pair-sim-mlp2-as-300-400-2400.json"

# filename = "logistics-pair-sim-mlp-as-300-1000-82000.json"
# with gfile.Open('/cns/od-d/home/xinranzhao/pair-sim-maps/'+filename, 'r') as f:
#   pair_sim_mapping_mlp_raw = json.load(f)

# pair_sim_mapping_mlp = []
# for item_scores in pair_sim_mapping_mlp_raw:
#   pair_sim_mapping_mlp.append(item_scores[:args.data_scope])

# print(len(pair_sim_mapping_mlp[0]))

## Partial Problem Generator in Pipeline

In [ ]:
# drive.SaveFile(filename=filename, data=json.dumps(new_output_collection))

if args.exec_based:
  plan_sim = "es"
  if args.init_exec:
    plan_sim += "i"
else:
  plan_sim = "as"

if args.object_centric_parse:
  plan_sim = "o"+plan_sim

# segs = ["pair-sim",str(args.test_scope),str(args.data_scope),plan_sim, args.plan_sim,args.client,str(args.num_demo),args.icl_sim,args.icl_sim_mode]

# filename = "-".join(segs)+".json"


# with gfile.Open('/cns/od-d/home/xinranzhao/pair-sim-maps/'+filename, 'w') as f:
#   json.dump(pair_sim_mapping, f)

# # with gfile.Open('/cns/od-d/home/xinranzhao/pair-sim-maps/'+filename, 'r') as f:
# #   pair_sim_mapping = json.load(f)

# segs = ["pair-sim-interval",str(args.test_scope),str(args.data_scope),plan_sim, args.plan_sim,args.client,str(args.num_demo),args.icl_sim,args.icl_sim_mode]

# filename = "-".join(segs)+".json"

# with gfile.Open('/cns/od-d/home/xinranzhao/pair-sim-maps/'+filename, 'w') as f:
#   json.dump(pair_sim_interval, f)

# with gfile.Open('/cns/od-d/home/xinranzhao/pair-sim-maps/'+filename, 'r') as f:
#   pair_sim_interval = json.load(f)

# filename = "pair-sim-100-28000-es-gecko-goldfish-1-plan-dynamic.json"
# filename = "pair-sim-100-28000-oes-gecko-goldfish-1-plan-static.json"
# with gfile.Open('/cns/od-d/home/xinranzhao/pair-sim-maps/'+filename, 'r') as f:
#   pair_sim_mapping = json.load(f)

In [ ]:
# to apply partial plan generator, we generate the partial inputs/targets for each test data first
input_template = """(:init
<init_state>
)
(:goal (and
<final_state>
))
)
"""

def goal_infer(state0,state1,args):
  goal_predicates = []
  for predicate in state1:
    # this is hard-coded for Block-World, need to be changed for further experiments
    #  note:
    # this line is tricky, if we use this filter, some intervals will return no goals
    # if we do not use the filter, as currently, the states may be undefined
    if predicate not in state0:
      if "on " in predicate or (not args.goal_infer_with_on):
        goal_predicates.append(predicate)

  return goal_predicates


def partial_plan_to_problem_generator(inputs, exec_plan, input_template, interval, args):
  # interval denotes the time step for the plan
  init_actions = convert_input_to_exec(inputs)
  states = [[action.replace("Adding ","") for action in init_actions]] # a list of the current states e.g., [(on b2 b5),(on b3 b4)]

  execs = exec_plan.split("Checking next happening") # execs[0] is the title'plan exec results:'
  # record the status as the plan runs

  # intervals are defined on the exections if the similarity is exec-based
  ori_init, ori_end = 0,0

  exec_action_count = 0

  for i,raw_exec in enumerate(execs[1:]):
    raw_actions = raw_exec.strip().splitlines()
    actions = []
    for action in raw_actions: # raw actions[0] is an indicator "time X"
      if "Adding" in action or "Deleting" in action:
        actions.append(action)
    current_state = [pred for pred in states[-1]]

    for action in actions:
    # Deleting (clear b3), Deleting (handempty), Adding (holding b3)
      segs = action.split()
      temp_predicate = " ".join(segs[1:])
      # print(current_state)
      # print(temp_predicate)
      # print("---")
      if segs[0] == "Deleting":
        current_state.remove(temp_predicate)
      elif segs[0] == "Adding":
        current_state.append(temp_predicate)

      if exec_action_count == interval[0]:
        ori_init = i

      if exec_action_count == interval[1]:
        ori_end = i
      exec_action_count += 1

    states.append(current_state)

  # if the interval is execution based, we map it back to the action level
  if args.exec_based:
    if ori_init == ori_end:
      ori_end = ori_init+1

    print("bug detected")
    interval = [ori_init, ori_end]

  # generate the template from the inputs
  output = ""
  for line in inputs.splitlines():
    if "init" in line:
      break
    output += line+"\n"
  output += input_template

  # interval is based on action, need to correct by add 1
  # interval = [interval[0]+1, interval[1]+1]

  if interval[0] > len(states)-1 or interval[0] == interval[1]:
    init_state = states[0]
  else:
    init_state = states[interval[0]]

  end_pos = len(states)-1 if interval[1] > len(states)-1 else interval[1]
  goal_state = states[end_pos]

  offset = 0

  goal_predicates = goal_infer(init_state, goal_state, args)

  while args.goal_infer_with_on and len(goal_predicates) == 0 and end_pos != -1:
    for i in range(len(states)-end_pos-2):
      offset += 1
      goal_state = states[end_pos+offset]
      goal_predicates = goal_infer(init_state, goal_state, args)

      if len(goal_predicates) != 0:
        break
    break


  output = output.replace("<init_state>","\n".join(init_state))
  output = output.replace("<final_state>", "\n".join(goal_predicates))

  return output, end_pos+offset

with gfile.Open('/cns/od-d/home/xinranzhao/val_results/train_v6.json', 'r') as f:
# #with gfile.Open('/cns/od-d/home/xinranzhao/val_results/dev_v6.json', 'r') as f:
  collected_exec_results = json.load(f)


# interval = [0,3]
# output,offset = partial_plan_to_problem_generator(data_lines[4]["inputs"], collected_exec_results[4], input_template, interval, args)
# print(data_lines[4]["inputs"])
# print("---")
# print(data_lines[4]["targets"].splitlines()[interval[0]:interval[1]])
# print("---")
# print(output)

## ICL Sweep

In [ ]:
# args.debug_mode = False

demo_template = """Please solve the problem:
<input>
Your plan as plain text without formatting:
<plan>
done.

"""
def post_gen_clean(output_collection):
  new_output_collection = []

  for item in output_collection:
    temp = [item[0], item[1], item[2].replace('done.', ' ').strip()]
    new_output_collection.append(temp)

  return new_output_collection


def partial_plan_gen_decorate(this_input, this_output, index, this_interval, args):
  new_input,new_end_pos = partial_plan_to_problem_generator(this_input, collected_exec_results[index], input_template, this_interval, args)
  if args.goal_infer_with_on:
    this_interval[1] = new_end_pos
  new_output = "\n".join(this_output.splitlines()[this_interval[0]:this_interval[1]])

  return new_input, new_output


def generate_with_retries(prompt, client):
  retry_count = 0
  while retry_count < MAX_RETRIES:
    try:
      response = client.generate(prompt)
      return response

    except Exception as e:
      retry_count += 1
      # print(f"Attempt {retry_count} failed for prompt '{prompt}': {e}")
      if retry_count < MAX_RETRIES:
        time.sleep(RETRY_INTERVAL)
      else:
        print(f"Max retries reached for prompt '{prompt}', skipping.")
        return None


def prompt_decorate(input, demo_data_lines, sim_index, item_scores, item_interval, args):
  # generate the prompt for a simple input
  prompt_init = "Please solve the problem:\n<input>\nYour plan as plain text without formatting:"

  if args.icl_sim_mode == "reverse":
    prompt = ""
  else:
    prompt = prompt_init.replace('<input>', input)

  count = 0

  if args.icl_sim_mode == "dynamic":

    mean = statistics.mean(item_scores)
    std = statistics.stdev(item_scores)
    offset = args.num_demo * std

    for i in sim_index:
      if item_scores[i] > mean+offset:
        this_input = demo_data_lines[i]["inputs"]
        this_plan = demo_data_lines[i]["targets"]
        if args.partial_plan_gen:
          this_interval = item_interval[i]
          this_input, this_plan = partial_plan_gen_decorate(this_input, this_plan, i, this_interval, args)
        demo_prompt = demo_template.replace("<input>",this_input).replace("<plan>",this_plan)
        prompt = demo_prompt +"\n"+ prompt
        count+=1

        if count > 20:
          break

  elif args.icl_sim_mode == "dcluster":
    for index in sim_index:
      this_input = demo_data_lines[index]["inputs"]
      this_plan = demo_data_lines[index]["targets"]

      demo_prompt = demo_template.replace("<input>",this_input).replace("<plan>",this_plan)
      prompt = demo_prompt +"\n"+ prompt

      count+=1

  elif args.icl_sim_mode in ["static", "reverse","cluster"] or count == 0 :
    for i in range(int(args.num_demo)):
      index = sim_index[i]
      this_input = demo_data_lines[index]["inputs"]
      this_plan = demo_data_lines[index]["targets"]

      demo_prompt = demo_template.replace("<input>",this_input).replace("<plan>",this_plan)

      if args.partial_plan_gen:
        this_interval = item_interval[index]
        # for AS+PPG
        this_input, this_plan = partial_plan_gen_decorate(this_input, this_plan, index, this_interval, args)
        demo_prompt += demo_template.replace("<input>",this_input).replace("<plan>",this_plan)
        # for AS+PPG (test)
        # this_input, this_plan = partial_plan_gen_decorate(input, this_plan, index, this_interval, args)

      if args.icl_sim_mode == "reverse":
        prompt = prompt +"\n" + demo_prompt
      else:
        prompt = demo_prompt +"\n"+ prompt
  else:
    print(f"Unknown icl_sim_mode: {args.icl_sim_mode}")

  if args.icl_sim_mode == "reverse":
    prompt = prompt+prompt_init.replace('<input>', input)

  return prompt, count


def icl_main(plans_df, test_df, args, pair_sim_map, pair_sim_interval, client):
  # running the demo

  output_collection = [] # for each instance, store input and plan
  data_lines = plans_df.to_dict(orient="records")
  test_lines = test_df.to_dict(orient="records")

  all_count = 0
  # decorate and run each example
  for i in tqdm(range(args.test_scope)):
    if args.icl_sim == "random":
      sim_index= range(args.data_scope)
      random.Random(42+i).shuffle(list(sim_index))
      item_scores = [0] * len(sim_index)
      item_interval = [0] * len(sim_index)
    else:
      sim_index = list((-np.array(pair_sim_map[i])).argsort())
      item_scores = pair_sim_map[i]

      if args.icl_sim_mode in ["dcluster","cluster"]:
        # three important features: search size, cluster num, per cluster max
        rough_search_size,n_clusters,per_cluster_sample_max = 100,1,1
        mean,std = statistics.mean(item_scores), statistics.stdev(item_scores)
        offset = 2

        if args.icl_sim_mode == "cluster":
          num_cluster = args.cluster_size
          n_clusters = args.num_demo
          rough_search_size = num_cluster*args.num_demo
          per_cluster_sample_max = args.per_cluster_sample_max

        elif args.icl_sim_mode == "dcluster":
          rough_search_size = 0
          offset = 2
          # for temp in range(40):
          for score in item_scores: # first sample
            if score > mean + offset*std:
              rough_search_size += 1
            # if rough_search_size > 100:
            #   rough_search_size = 0 # empirically, around 100 makes the best score
            #   offset += 0.1
            # else:
            #   break

          if rough_search_size > 80:
            rough_search_size = 80
            offset += 2

          # if args.debug_mode:
          #   print("rough_search_size: ", rough_search_size)

          n_clusters = int(rough_search_size**0.25+1)*args.num_demo
          # if args.debug_mode:
          #   print("n_clusters: ", n_clusters)

        # use clustering to sample different demos, 4 is an arbitary number
        rough_indexes = sim_index[:rough_search_size]
        rough_df = plans_df.iloc[rough_indexes,:]
        rough_pair_sim_mapping = find_pair_sim_mapping_sq(rough_df, args)
        pair_dist_map = []

        for item in rough_pair_sim_mapping:
          temp = []
          for x in item:
            if x == 0:
              x = 1e-10
            temp.append(1/x)
          pair_dist_map.append(temp)

        clustering = AgglomerativeClustering(n_clusters=n_clusters, affinity='precomputed', linkage='complete')
        clustering.fit(pair_dist_map)
        cluster_labels = clustering.labels_
        new_sim_index = []
        visited_clusters = defaultdict(int)
        for temp_i in range(rough_search_size):
          index = sim_index[temp_i]
          if args.icl_sim_mode == "cluster":
            if visited_clusters[cluster_labels[temp_i]] <= per_cluster_sample_max:
              new_sim_index.append(index)
              visited_clusters[cluster_labels[temp_i]] += 1
          if args.icl_sim_mode == "dcluster":
            # at least one example, and not miss 2std examples
            if visited_clusters[cluster_labels[temp_i]] <= per_cluster_sample_max or item_scores[index] >= (mean+(offset+1)*std):
              # at least one or two is there, or it is significantly good
              new_sim_index.append(index)
              visited_clusters[cluster_labels[temp_i]] += 1
        # if args.debug_mode:
        #   print("sample size: ", len(new_sim_index))

        # we change the original sim_index to the new index
        # print(sim_index[:args.num_demo], new_sim_index[:args.num_demo])
        sim_index = new_sim_index
        real_count = len(new_sim_index)


      if args.partial_plan_gen:
        item_interval = pair_sim_interval[i]
      else:
        item_interval = []

    # for AS+PPG test
    # sim_index = [i for x in range(args.num_demo)]
    real_count = len(sim_index)
    prompt, count = prompt_decorate(test_lines[i]["inputs"], data_lines, sim_index, item_scores, item_interval,args)
    all_count += real_count

    # response = client.generate(prompt)
    # response = m.query(prompt)

    if args.debug_mode:
      output = ""
    else:
      response = generate_with_retries(prompt, client)
      output = model_client.as_text(response)


    output_collection.append([test_lines[i]["inputs"], test_lines[i]["targets"], output])

  # store the information to drive with args annotaion
  if args.debug_mode:

    print("count:", all_count/args.test_scope)

  return output_collection


pair_sim_map = []

# obj_d = as
# oas
# m = tm.models.Gemini()
# to do as-100
for icl_sim in ["asr"]:#"random","plan",,"input"
  args.icl_sim = icl_sim
  print(args.icl_sim_mode)
  # pair_sim_map, pair_sim_interval, pair_sim_mapping_input = None, None, None

  if args.icl_sim == "plan":
    pair_sim_map = pair_sim_mapping
  elif args.icl_sim == "input":
    pair_sim_map = pair_sim_mapping_input
  elif args.icl_sim == "mlp":
    pair_sim_map = pair_sim_mapping_mlp
  elif args.icl_sim == "bpe":
    pair_sim_map = pair_sim_mapping_bpe_merged
  elif args.icl_sim == "asr":
    pair_sim_map = pair_sim_mapping_asr

  if not args.partial_plan_gen:
    pair_sim_interval = []

  # for num_demo in [1,2,3]:
  for num_demo in [4,10,20,40,100]:#
    args.num_demo = num_demo
    output_collection = icl_main(plans_df, test_df, args, pair_sim_map, pair_sim_interval, client)

    new_output_collection = post_gen_clean(output_collection)

    # print(new_output_collection[0][-1])
    if args.exec_based:
      plan_sim = "es"
      if args.init_exec:
        plan_sim += "i"
      else:
        plan_sim += "v"
    else:
      plan_sim = "as"

    if args.object_centric_parse:
      plan_sim = "o"+plan_sim

    if args.partial_plan_gen:
      plan_sim = plan_sim+"-ppgid"

    segs = ["dev",str(args.test_scope),str(args.data_scope),plan_sim, args.plan_sim,args.client,str(args.num_demo),args.icl_sim,args.icl_sim_mode]

    if args.icl_sim_mode == "cluster":
      segs = ["dev",str(args.test_scope),str(args.data_scope),plan_sim, args.plan_sim,args.client,str(args.num_demo),args.icl_sim,args.icl_sim_mode+str(args.cluster_size)+str(args.per_cluster_sample_max)]

    filename = "-".join(segs)+".json"

    # drive.SaveFile(filename=filename, data=json.dumps(new_output_collection))
    if not args.debug_mode:
      with gfile.Open('/cns/od-d/home/xinranzhao/dev-300/tetris/iter1-'+filename, 'w') as f:
        json.dump(new_output_collection, f)

static


  0%|          | 0/300 [00:00<?, ?it/s]

In [ ]:
args.debug_mode = False

for icl_sim in ["asr"]:#"random","plan",,"input"
  args.icl_sim = icl_sim
  args.icl_sim_mode = "dcluster"
  print(args.icl_sim_mode)
  # pair_sim_map, pair_sim_interval, pair_sim_mapping_input = None, None, None

  if args.icl_sim == "plan":
    pair_sim_map = pair_sim_mapping
  elif args.icl_sim == "input":
    pair_sim_map = pair_sim_mapping_input
  elif args.icl_sim == "mlp":
    pair_sim_map = pair_sim_mapping_mlp
  elif args.icl_sim == "bpe":
    pair_sim_map = pair_sim_mapping_bpe_merged
  elif args.icl_sim == "asr":
    pair_sim_map = pair_sim_mapping_asr

  if not args.partial_plan_gen:
    pair_sim_interval = []

  for num_demo in [1,2,3,4]:
  # for num_demo in [1,4,10,20,40]:#
    args.num_demo = num_demo
    output_collection = icl_main(plans_df, test_df, args, pair_sim_map, pair_sim_interval, client)

    new_output_collection = post_gen_clean(output_collection)

    # print(new_output_collection[0][-1])
    if args.exec_based:
      plan_sim = "es"
      if args.init_exec:
        plan_sim += "i"
      else:
        plan_sim += "v"
    else:
      plan_sim = "as"

    if args.object_centric_parse:
      plan_sim = "o"+plan_sim

    if args.partial_plan_gen:
      plan_sim = plan_sim+"-ppgid"

    segs = ["dev",str(args.test_scope),str(args.data_scope),plan_sim, args.plan_sim,args.client,str(args.num_demo),args.icl_sim,args.icl_sim_mode]

    if args.icl_sim_mode == "cluster":
      segs = ["dev",str(args.test_scope),str(args.data_scope),plan_sim, args.plan_sim,args.client,str(args.num_demo),args.icl_sim,args.icl_sim_mode+str(args.cluster_size)+str(args.per_cluster_sample_max)]

    filename = "-".join(segs)+".json"

    # drive.SaveFile(filename=filename, data=json.dumps(new_output_collection))
    if not args.debug_mode:
      with gfile.Open('/cns/od-d/home/xinranzhao/dev-300/logistics/iter1-'+filename, 'w') as f:
        json.dump(new_output_collection, f)

dcluster


  0%|          | 0/300 [00:00<?, ?it/s]

KeyboardInterrupt: 

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

In [ ]:
# # one shot sanity test
# filename = "logistics-pair-sim-mlp-as-300-1000-82000.json"
# with gfile.Open('/cns/od-d/home/xinranzhao/pair-sim-maps/'+filename, 'r') as f:
#   pair_sim_mapping_mlp = json.load(f)
# #
# print(len(pair_sim_mapping_mlp[0]))

# for icl_sim in ["mlp"]:#"random","plan",,"input"
#   args.icl_sim = icl_sim
#   # pair_sim_map, pair_sim_interval, pair_sim_mapping_input = None, None, None

#   if args.icl_sim == "plan":
#     pair_sim_map = pair_sim_mapping
#   elif args.icl_sim == "input":
#     pair_sim_map = pair_sim_mapping_input
#   elif args.icl_sim == "mlp":
#     pair_sim_map = pair_sim_mapping_mlp
#   elif args.icl_sim == "bpe":
#     pair_sim_map = pair_sim_mapping_bpe_merged

#   if not args.partial_plan_gen:
#     pair_sim_interval = []

#   # for num_demo in [2,3,4,5]:
#   for num_demo in [10,20,40]:#
#     args.num_demo = num_demo
#     output_collection = icl_main(plans_df, test_df, args, pair_sim_map, pair_sim_interval, client)

#     new_output_collection = post_gen_clean(output_collection)

#     # print(new_output_collection[0][-1])
#     if args.exec_based:
#       plan_sim = "es"
#       if args.init_exec:
#         plan_sim += "i"
#       else:
#         plan_sim += "v"
#     else:
#       plan_sim = "as"

#     if args.object_centric_parse:
#       plan_sim = "o"+plan_sim

#     if args.partial_plan_gen:
#       plan_sim = plan_sim+"-ppgid"

#     segs = ["dev",str(args.test_scope),str(args.data_scope),plan_sim, args.plan_sim,args.client,str(args.num_demo),args.icl_sim,args.icl_sim_mode]

#     if args.icl_sim_mode == "cluster":
#       segs = ["dev",str(args.test_scope),str(args.data_scope),plan_sim, args.plan_sim,args.client,str(args.num_demo),args.icl_sim,args.icl_sim_mode+str(args.cluster_size)+str(args.per_cluster_sample_max)]

#     filename = "-".join(segs)+".json"

#     # drive.SaveFile(filename=filename, data=json.dumps(new_output_collection))
#     if not args.debug_mode:
#       with gfile.Open('/cns/od-d/home/xinranzhao/dev-300/logistics/'+filename, 'w') as f:
#         json.dump(new_output_collection, f)

# # check prompt
# # args.num_demo = 1

# # def prompt_decorate(input, demo_data_lines, sim_index, args):
# #   # generate the prompt for a simple input
# #   prompt = "TEST:Please solve the problem:\n<input>\nYour plan as plain text without formatting:"
# #   prompt = prompt.replace('<input>', input)

# #   for i in range(args.num_demo):
# #     index = sim_index[i]
# #     print(index)
# #     demo_prompt = demo_template.replace("<input>",demo_data_lines[index]["inputs"]).replace("<plan>",demo_data_lines[index]["targets"])
# #     prompt = demo_prompt +"\n"+ prompt

# #   print(prompt)
# #   return prompt

# # data_lines = plans_df.to_dict(orient="records")
# # test_lines = test_df.to_dict(orient="records")
# # a = prompt_decorate(test_lines[0]["inputs"], test_lines, [0], args)

82000


  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

KeyboardInterrupt: 

# PDDL VAL Execution

In [ ]:
# imported from google3/learning/brain/research/pagi/multiagent/eval_utils.py
# PDDL_EXECUTABLE = '/cns/od-d/home/pagi/planning/VAL/validate'
PDDL_EXECUTABLE = './VAL/validate'
# PDDL_EXECUTABLE = '/google_src/files/head/depot/google3/experimental/pagi/planning/PDDL/VAL/validate'



def call_pddl_validator(domain: str, instance: str, plan: str) -> bool:
  """Run the PDDL validation tool.

  Args:
    domain: problem domain file in pddl
    instance: problem instance file in pddl
    plan: plan file in htn

  Returns:
    Plan verification result.
  """
  # Ensure executable.
  gfile.SetMode(PDDL_EXECUTABLE, 0o755) #-a
  # val_exe = f'{PDDL_EXECUTABLE} -v %s %s %s' % (
  #     domain,
  #     instance,
  #     plan,
  # )
  val_exe = f'{PDDL_EXECUTABLE} -v %s %s %s' % (
      domain,
      instance,
      plan,
  )

  response = subprocess.run(
      val_exe, shell=True, check=False, capture_output=True
  )
  print(response)

  return (
      response is not None and b'Plan executed successfully' in response.stdout
  )

def pddl_validate_plan(
    plan: str,
    instance: str,
    domain_path: str,
    # cmd_args: str = '-a',
    debug: bool = True,
) -> float:
  """PDDL validation given a generated plan.

  Args:
    plan: Plan to evaluate with PDDL validator.
    instance: String representing task instance.
    domain_path: Path to domain.
    reward_value_map:  Mapping of validator outputs to rewards.
    debug:  Specify whether or not to output debug statements.

  Returns:
  """
  plan_path = os.path.join('./', 'plan.pddl')
  with open(plan_path, 'w') as f:
    f.write(plan)
  instance_path = os.path.join('./', 'instance.pddl')
  with open(instance_path, 'w') as f:
    f.write(instance)

  result = call_pddl_validator(
      domain_path,
      instance_path,
      plan_path,
  )

  return result

# this command does not work locally
# pddl_validate_plan(ex_plan, ex_input, domain_path="./generated_domain.pddl")

In [ ]:
prompt_template = """
Please solve the problem:
<input>
Your plan as plain text without formatting:
<plan>
done.
"""


# from colabtools.drive import mount

# drive.SaveFile(filename='test.txt', data=prompt_template)
# drive.MoveFileToFolder(file_id='test.txt', folder_id='1HDmk8NlcukqqRlTf6-N17K18rPZ3eRHs')
# print(colabtools.notebookinfo.get_google3_base())
# print(colabtools.notebookinfo.get_url())
# print(os.getcwd())
# mount('/conltent/gdrive')

# Learning to score and sample plans from test problems

In [ ]:
# using model-based methods to empirically calculate problem to plan scores
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split



# Potential Models:
# 1. Combined: uni-score MLP + Gemini Filter:
#   Uni-score MLP: we train a MLP with problem/plan embeddings to assign a score for each plan, with plan similarity as the signal
#   Gemini Filter: after getting the high score plans, we utilize Gemini to sample which subpart is useful, with ICL
#   [Note] Uni-score MLP can be changed to any problem-to-problem similarity method, e.g., embeddings
# 2. Sinlge: Seq-2-Seq Model:
#   S2S: train a ULM model that uses instruction and input (problem, demo problem and plan) to generate a list of useful plan indexes

def prepare_emb(plans_df, test_df, em, args):
  # convert the input to embs
  data_lines = plans_df.to_dict(orient="records")
  test_lines = test_df.to_dict(orient="records")

  print("Prepare embeddings...")
  data_inputs_embs = convert_tm_embedding(data_lines[:args.data_scope], em, args)
  test_inputs_embs = convert_tm_embedding(test_lines, em, args)
  data_targets_embs = convert_tm_embedding(data_lines[:args.data_scope], em, args, keyword="targets")
  # test_targets_embs = convert_tm_embedding(test_lines, em, args, keyword="targets")

  print("Prepare regression label...")
  args.test_scope = data_scope
  train_exec, test_exec = None, None
  pair_sim_mapping, pair_sim_interval = find_pair_sim_mapping_main(plans_df, plans_df, train_exec, test_exec, args)
  args.test_scope = test_scope
  test_pair_sim_mapping, pair_sim_interval = find_pair_sim_mapping_main(plans_df, test_df, train_exec, test_exec, args)
  print("Prepare Training data...")
  # X,y = [],[]
  X_train, X_test, y_train, y_test = [],[],[],[]

  for ind1 in range(len(data_inputs_embs)):
    for ind2 in range(len(data_inputs_embs)):
      if ind1 != ind2:
        temp = []
        temp.extend(data_inputs_embs[ind1])
        temp.extend(data_targets_embs[ind1])
        temp.extend(data_inputs_embs[ind2])
        X_train.append(temp)
        y_train.append(pair_sim_mapping[ind2][ind1])

  # regr = MLPRegressor(hidden_layer_sizes=(100,100,50),random_state=42, max_iter=50,verbose=True).fit(X_train, y_train)
  regr = MLPRegressor(random_state=42, max_iter=25,verbose=True).fit(X_train, y_train)
  #regr = MLPRegressor(hidden_layer_sizes=(100),random_state=42, max_iter=25,verbose=True).fit(X_train, y_train)
  X_train,y_train = [],[]

  for ind1 in range(len(data_inputs_embs)):
    for ind2 in range(args.test_scope):
      temp = []
      temp.extend(data_inputs_embs[ind1])
      temp.extend(data_targets_embs[ind1])
      temp.extend(test_inputs_embs[ind2])
      X_test.append(temp)
      y_test.append(test_pair_sim_mapping[ind2][ind1])

  # X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=42)


  regr.predict(X_test[:2])
  score = regr.score(X_test, y_test)
  print(score)
  return regr


# data_scope = 800
# test_scope = 100

# args.data_scope = data_scope
# args.test_scope = test_scope
# regr = prepare_emb(plans_df, test_df, em, args)

In [ ]:
# paras = regr.get_params()
# print(paras.keys())

# with gfile.Open('/cns/od-d/home/xinranzhao/pair-sim-maps/regr-2_400_params.json', mode='w') as f:
#   json.dump(paras, f)

with gfile.Open('/cns/od-d/home/xinranzhao/pair-sim-maps/regr-2_400_params.json', mode='r') as f:
  paras = json.load(f)

regr_reload =  MLPRegressor(random_state=42, max_iter=100, verbose=True)
regr_reload.set_params(**paras)
print(type(paras))

<class 'dict'>


In [ ]:
data_lines = plans_df.to_dict(orient="records")
test_lines = test_df.to_dict(orient="records")

# data_inputs_embs = convert_tm_embedding(data_lines, em, args)
# data_targets_embs = convert_tm_embedding(data_lines, em, args, keyword="targets")
# with gfile.Open('/cns/od-d/home/xinranzhao/pair-sim-maps/block_train_v6_inputs_embs.json', mode='w') as f:
#   json.dump([x.tolist() for x in data_inputs_embs], f)

# with gfile.Open('/cns/od-d/home/xinranzhao/pair-sim-maps/block_train_v6_inputs_embs.json', mode='r') as f:
#   data_inputs_embs = json.load(f)

# test_inputs_embs = convert_tm_embedding(test_lines, em, args)
# test_targets_embs = convert_tm_embedding(test_lines, em, args, keyword="targets")
# with gfile.Open('/cns/od-d/home/xinranzhao/pair-sim-maps/block_dev_v6_targets_embs.json', mode='w') as f:
#   json.dump([x.tolist() for x in test_targets_embs], f)

# Experiments on Natural Planning

In [ ]:
trip_train = os.path.join(DATA_DIR, "natural_planning_amended_train_set.jsonl")
trip_dev = os.path.join(DATA_DIR, "natural_planning_amended_dev_set.jsonl")

# trip_train = os.path.join(DATA_DIR, "calendar_planning_amended_train_set.jsonl")
# trip_dev = os.path.join(DATA_DIR, "calendar_planning_amended_dev_set.jsonl")

data_lines = []
with gfile.Open(trip_train, mode='r') as f:
  for l in f.readlines():
    data_lines.append(json.loads(l))

test_lines = []
with gfile.Open(trip_dev, mode='r') as f:
  for l in f.readlines():
    test_lines.append(json.loads(l))

print(len(data_lines))
print(len(test_lines))

1344
336


In [ ]:
# @title Validator
def hour_to_num(hour: str) -> float:
  ret = 0
  if hour.split(':')[1] == '30':
    ret += 0.5
  return ret + float(hour.split(':')[0])

def parse_calendar_response(response: str, task_parse=False) -> Tuple[str, float, float]:
  # time_str = response.split('\n')[0].split(':')
  time_str = re.findall(r'[A-Za-z]+, [0-9]+:[0-9]+ - [0-9]+:[0-9]+', response)
  if task_parse:
    # example ['9:30 - 10:30', '12:30 - 13:00', '15:30 - 16:00']
    time_collection = []
    lines = response.split('\n')
    for line in lines:
      dis_time_str = [] # e.g., 9.5, 10.0 10.5
      time_str = re.findall(r'[0-9]+:[0-9]+ - [0-9]+:[0-9]+', line)
      if len(time_str) > 0:
        for interval in time_str:
          interval = interval.replace(" ","")
          start_hour, end_hour = interval.split('-')
          start_hour, end_hour = hour_to_num(start_hour), hour_to_num(end_hour)
          start_hour, end_hour = int(start_hour*10), int(end_hour*10)
          dis_time_str.extend(list(range(start_hour, end_hour, 5)))
          dis_time_str.append(end_hour)
        time_collection.extend(dis_time_str)
    return time_collection

  if not time_str:
    return '', -1, -1
  time_str = time_str[0]
  day, hour = time_str.split(',')[0].strip(), time_str.split(',')[1].strip()
  start_hour, end_hour = hour.split('-')[0].strip(), hour.split('-')[1].strip()
  return day, hour_to_num(start_hour), hour_to_num(end_hour)


def compute_calendarplan_accuracy(response: str,
                                  solution: str) -> Tuple[int, int]:
  """Compute hard/soft accuracy on CalendarPlan task."""
  # print('compute_accuracy, responses', response)
  # print('compute_accuracy, solutions', solution)
  day_correct, all_correct = 0, 0

  r_day, r_start_hour, r_end_hour = parse_calendar_response(response)
  s_day, s_start_hour, s_end_hour = parse_calendar_response(solution)
  if r_day == s_day:
    day_correct += 1
  if r_start_hour == s_start_hour and r_end_hour == s_end_hour:
    all_correct += 1

  return day_correct, all_correct


def parse_response(response):
    pattern_visit = r"\d+-\d+"
    pattern_flight = r".*Day (\d+).*from (\w+) to (\w+)"
    pattern_days = r"European cities for (\d+) days"

    days, flights, flight_days = [], [], []
    total_days = None
    for piece in response.split('\n'):
      days_match = re.findall(pattern_days, piece)
      if days_match:
        total_days = int(days_match[0])

      visit_match = re.findall(pattern_visit, piece)
      if visit_match:
        days.append(visit_match[0])
        end_day = int(visit_match[0].split('-')[1])
        # Reach the end of the plan, stop to avoid parsing alternative plans.
        if end_day == total_days: break
      flight_match = re.findall(pattern_flight, piece)
      if flight_match:
        flights.append(flight_match[0])

    visit_cities, parsed_plan = [], []
    # print(flights)
    for flight_day, begin_city, end_city in flights:
      flight_days.append(int(flight_day))
      if not visit_cities:
        visit_cities.append(begin_city)
        visit_cities.append(end_city)
      else:
        last_city = visit_cities[-1]
        # if begin_city != last_city:
        #   print('='*40)
        #   print(f"Parsing error, the flights are not consecutive: {begin_city} vs {last_city} in flights {flights}.")
        #   print(response)
        #   print('='*40)
        #   print('\n')
        visit_cities.append(end_city)

    if not days or not flights or not visit_cities:
      # print(f'Parsing failed for response: {response} with days: {days}, flights: {flights}, visit_cities: {visit_cities}.')
      return []
    last_day = int(days[-1].split('-')[1])
    flight_days = [1] + flight_days + [last_day]
    # print(flight_days)
    # print(visit_cities)
    for i in range(len(visit_cities)):
      city_stay = flight_days[i+1] - flight_days[i] + 1
      city = visit_cities[i]
      parsed_plan.append((city, city_stay))

    return parsed_plan


def find_trip_pair_sim_mapping_main(data_lines, test_lines, args):
  pair_sim_mapping = []
  pair_sim_interval = []

  if args.natural_task == "trip":
    parsed_plans = [parse_response(item["solution"]) for item in data_lines]
    test_plans = [parse_response(item["solution"]) for item in test_lines]

    def trip_action_parse(parsed_plan):
      output = []
      for item in parsed_plan:
        for i in range(item[1]):
          output.append(item[0])

      return [output]

    action_lists = [trip_action_parse(parsed_plan) for parsed_plan in parsed_plans]
    test_action_lists = [trip_action_parse(parsed_plan) for parsed_plan in test_plans]

  elif args.natural_task == "calendar":
    action_lists = [parse_calendar_response(item["task"].replace('to', '-'),task_parse=True) for item in data_lines]
    test_action_lists = [parse_calendar_response(item["task"].replace('to', '-'),task_parse=True) for item in test_lines]

  else:
    print(f'Unknown natural task: {args.natural_task}')
    action_lists = []
    test_action_lists = []

  # design choice: sampling 100 data points, 30.72 it/s required for each pair
  for i, action_list0 in enumerate(tqdm(test_action_lists[:args.test_scope])):
    this_pair_sim_mapping = []
    this_pair_sim_interval = []

    for j, action_list1 in enumerate(action_lists[:args.data_scope]):
      # if we assume a undirectioned metric
      if args.undir_metric and j < i:
        this_pair_sim_mapping.append(pair_sim_mapping[j][i])
        continue

      common_list_collection,common_len_collection = [],[]
      common_list = [] # dummy
      common_a1_interval = [0,0]
      max_common_len = 0

      for a0 in action_list0:
        this_common_list, this_common_len = [],[]
        for a1 in action_list1:
          common_list, common_a1_interval = longest_common_actions(a0, a1, args)
          common_len = len(common_list)
          # consider the case where there are multiple common_list
          if args.lca_with_reoccur:
            count0 = " ".join(a0).count(" ".join(common_list))
            count1 = " ".join(a1).count(" ".join(common_list))
            factor = max(count0, count1)
            common_len = factor * common_len

          # design choice: how and if we should penalize the length
          if len(a0) != 0:
            common_len = common_len**2 / (len(a0) * len(a1))
          else:
            common_len = 0

          if common_len > max_common_len:
            max_common_len = common_len

          this_common_list.append(common_list)
          this_common_len.append(common_len)

        common_list_collection.append(this_common_list)
        common_len_collection.append(this_common_len)

      if args.print_detail:
        print(len(common_list_collection))
        print(len(common_list_collection[0]))
        print(common_list_collection[0])

      # organize with mean(max(per object action series))
      sim_score = 0
      if args.score_conv == "mean-max":
        temp_score = [max(x) for x in common_len_collection]
        sim_score = sum(temp_score) / len(temp_score)

      if args.score_conv == "max-max":
        sim_score = max_common_len

      this_pair_sim_mapping.append(sim_score)
      this_pair_sim_interval.append(common_a1_interval)

    pair_sim_mapping.append(this_pair_sim_mapping)
    pair_sim_interval.append(this_pair_sim_interval)

  return pair_sim_mapping, pair_sim_interval

def find_trip_pair_sim_mapping_sq(data_lines, args):
  # for clustering based method
  pair_sim_mapping = []
  action_lists = []

  if args.natural_task == "trip":
    parsed_plans = [parse_response(item["solution"]) for item in data_lines]

    def trip_action_parse(parsed_plan):
      output = []
      for item in parsed_plan:
        for i in range(item[1]):
          output.append(item[0])

      return [output]

    action_lists = [trip_action_parse(parsed_plan) for parsed_plan in parsed_plans]

  elif args.natural_task == "calendar":
    action_lists = [parse_calendar_response(item["task"].replace('to', '-'),task_parse=True) for item in data_lines]

  for i, action_list0 in enumerate(action_lists):
    this_pair_sim_mapping = []

    for j, action_list1 in enumerate(action_lists):
      # if we assume a undirectioned metric
      if args.undir_metric and j < i:
        this_pair_sim_mapping.append(pair_sim_mapping[j][i])
        continue

      common_list_collection,common_len_collection = [],[]
      common_list = [] # dummy
      common_a1_interval = [0,0]
      max_common_len = 0

      for a0 in action_list0:
        this_common_list, this_common_len = [],[]
        for a1 in action_list1:
          common_list, common_a1_interval = longest_common_actions(a0, a1, args)
          common_len = len(common_list)
          # consider the case where there are multiple common_list
          if args.lca_with_reoccur:
            count0 = " ".join(a0).count(" ".join(common_list))
            count1 = " ".join(a1).count(" ".join(common_list))
            factor = max(count0, count1)
            common_len = factor * common_len

          # design choice: how and if we should penalize the length
          common_len = common_len**2 / (len(a0) * len(a1))
          if common_len > max_common_len:
            max_common_len = common_len

          this_common_list.append(common_list)
          this_common_len.append(common_len)

        common_list_collection.append(this_common_list)
        common_len_collection.append(this_common_len)

      # organize with mean(max(per object action series))
      sim_score = 0
      if args.score_conv == "mean-max":
        temp_score = [max(x) for x in common_len_collection]
        sim_score = sum(temp_score) / len(temp_score)

      if args.score_conv == "max-max":
        sim_score = max_common_len

      this_pair_sim_mapping.append(sim_score)

    pair_sim_mapping.append(this_pair_sim_mapping)

  return pair_sim_mapping


def compute_accuracy(parsed_plan, cities, durations):

    if isinstance(cities, str):
      stays = [x for x in cities.split('**') if x]
    else:
      stays = cities
    if isinstance(durations, str):
      days = [int(x) for x in durations.split('**') if x]
    else:
      days = durations
    num_stays = min(len(stays), len(parsed_plan))
    num_match = 0
    for i in range(num_stays):
      if stays[i] == parsed_plan[i][0] and days[i] == parsed_plan[i][1]:
        num_match += 1
      else:
        break
    soft_score = num_match / len(stays)
    hard_score = 0.0 if soft_score < 1.0 else 1.0
    return (soft_score, hard_score)

print(test_lines[0].keys())
# print(compute_calendarplan_accuracy(test_lines[0]["solution"], test_lines[0]["solution"]))

# print(test_lines[0]["task"])
# print("------------------")
# print(test_lines[2]["solution"])
# print(parse_calendar_response(test_lines[-1]["task"].replace('to', '-'),task_parse=True))

dict_keys(['num_cities', 'cities', 'constraints', 'durations', 'total_days', 'task', 'split', 'solution'])


In [ ]:
# print(pair_sim_mapping[0])

In [ ]:
def create_single_trip_prompt(input_target, include_target=True):
  prompt = f"""Please solve the problem:
{input_target['task']}

"""

  if include_target:
    prompt += f"""
{input_target['solution']}
done.

"""
  return prompt


def trip_prompt_decorate(test_example, demo_data_lines, sim_index, item_scores, item_interval, args):
  # generate the prompt for a simple input

  if args.icl_sim_mode == "reverse":
    prompt = ""
  else:
    prompt = create_single_trip_prompt(test_example, include_target=False)

  count = 0


  if args.icl_sim_mode in ["dcluster","cluster"]:

      # three important features: search size, cluster num, per cluster max
      rough_search_size,n_clusters,per_cluster_sample_max = 100,1,1
      mean,std = statistics.mean(item_scores), statistics.stdev(item_scores)
      offset = 2

      if args.icl_sim_mode == "cluster":
        num_cluster = args.cluster_size
        n_clusters = args.num_demo
        rough_search_size = num_cluster*args.num_demo
        per_cluster_sample_max = args.per_cluster_sample_max

      elif args.icl_sim_mode == "dcluster":
        rough_search_size = 0
        offset = 2
        # for temp in range(40):
        for score in item_scores: # first sample
          if score > mean + offset*std:
            rough_search_size += 1

        if rough_search_size > 80:
          rough_search_size = 80
          offset += 2

        if args.debug_mode:
          print("rough_search_size: ", rough_search_size)

        n_clusters = int(rough_search_size**0.25+1)*args.num_demo
        if args.debug_mode:
          print("n_clusters: ", n_clusters)

      # use clustering to sample different demos, 4 is an arbitary number
      rough_indexes = sim_index[:rough_search_size]
      rough_data_lines = []
      for tmp in rough_indexes:
        rough_data_lines.append(demo_data_lines[tmp])
      rough_pair_sim_mapping = find_trip_pair_sim_mapping_sq(rough_data_lines, args)
      pair_dist_map = []

      for item in rough_pair_sim_mapping:
        temp = []
        for x in item:
          if x == 0:
            x = 1e-10
          temp.append(1/x)
        pair_dist_map.append(temp)

      clustering = AgglomerativeClustering(n_clusters=n_clusters, affinity='precomputed', linkage='complete')
      clustering.fit(pair_dist_map)
      cluster_labels = clustering.labels_
      new_sim_index = []
      visited_clusters = defaultdict(int)
      for temp_i in range(rough_search_size):
        index = sim_index[temp_i]
        if args.icl_sim_mode == "cluster":
          if visited_clusters[cluster_labels[temp_i]] <= per_cluster_sample_max:
            new_sim_index.append(index)
            visited_clusters[cluster_labels[temp_i]] += 1
        if args.icl_sim_mode == "dcluster":
          # at least one example, and not miss 2std examples
          if visited_clusters[cluster_labels[temp_i]] <= per_cluster_sample_max or item_scores[index] >= (mean+(offset+1)*std):
            # at least one or two is there, or it is significantly good
            new_sim_index.append(index)
            visited_clusters[cluster_labels[temp_i]] += 1
      if args.debug_mode:
        print("sample size: ", len(new_sim_index))

      # we change the original sim_index to the new index
      # print(sim_index[:args.num_demo], new_sim_index[:args.num_demo])
      sim_index = new_sim_index

      if args.debug_mode:
        print("Sample Size: ", len(new_sim_index))

      count += len(new_sim_index)

      for i in sim_index:
        index = sim_index[i]
        demo_prompt = create_single_trip_prompt(demo_data_lines[index])
        prompt = demo_prompt +"\n"+ prompt

  if args.icl_sim_mode in ["dynamic"]:

    mean = statistics.mean(item_scores)
    std = statistics.stdev(item_scores)
    offset = args.num_demo * std

    for i in sim_index:
      if item_scores[i] > mean+offset:
        demo_prompt = create_single_trip_prompt(demo_data_lines[i])
        prompt = demo_prompt +"\n"+ prompt
        count+=1

        if count > 20:
          break

  if args.icl_sim_mode == "static" or args.icl_sim_mode == "reverse" or count == 0 :
    for i in range(args.num_demo):
      index = sim_index[i]
      demo_prompt = create_single_trip_prompt(demo_data_lines[index])

      if args.icl_sim_mode == "reverse":
        prompt = prompt +"\n" + demo_prompt
      else:
        prompt = demo_prompt +"\n"+ prompt

  if args.icl_sim_mode == "reverse":
    prompt = prompt + create_single_trip_prompt(test_example, include_target=False)

  return prompt, count


def trip_icl_main(data_lines, test_lines, args, pair_sim_map, pair_sim_interval, client):
  # running the demo, for trip, we can run the evaluation directly

  output_collection = [] # for each instance, store input and plan
  city_stats = {}

  all_count, correct, valid, total = 0, 0, 0, 0
  # decorate and run each example

  for i in tqdm(range(args.test_scope)):
    if args.icl_sim == "random":
      sim_index= range(args.data_scope)
      random.Random(42+i).shuffle(list(sim_index))
      item_scores = [0] * len(sim_index)
      item_interval = [0] * len(sim_index)
    else:
      sim_index = list((-np.array(pair_sim_map[i])).argsort())
      item_scores = pair_sim_map[i]
      if args.partial_plan_gen:
        item_interval = pair_sim_interval[i]
      else:
        item_interval = []

    prompt, count = trip_prompt_decorate(test_lines[i], data_lines, sim_index, item_scores, item_interval,args)

    all_count += count

    # response = client.generate(prompt)
    response = generate_with_retries(prompt, client)
    # response = m.query(prompt)

    output = model_client.as_text(response)
    output_collection.append([test_lines[i]["task"], test_lines[i]["solution"], output])

    if args.natural_task == "trip":
      score = compute_accuracy(parse_response(output), test_lines[i]["cities"], test_lines[i]["durations"])
    elif args.natural_task == "calendar":
      score = compute_calendarplan_accuracy(output,test_lines[i]["solution"])
    else:
      raise ValueError(f"Unknown natural task: {args.natural_task}")
      score = [0,0]

    soft_scores = score[0]
    hard_scores = score[1]

    entry_str = "num_cities" if args.natural_task == "trip" else "num_people"

    max_cities = test_lines[i][entry_str]
    stat = city_stats.get(max_cities, [0,0,0])

    if soft_scores == 1:
      valid += 1
    if hard_scores == 1:
      correct += 1

    new_stat = [stat[0] + int(hard_scores == 1),stat[1] + int(soft_scores == 1), stat[2] + 1 ]
    city_stats[max_cities] = new_stat
    total += 1

  # store the information to drive with args annotaion
  if args.icl_sim_mode == "dynamic":
    print(all_count)

  print("Valid:",valid)
  print("Correct:",correct)
  print("Total:",total)
  print("City Stats:",city_stats)

  return output_collection

In [ ]:
# start in-context learning
# VAL will be run separately on the cloudtop

data_scope = 1300
test_scope = 300

args = SudoArgs()
args.data_scope = data_scope
args.test_scope = test_scope
args.print_detail = False # for debugging and visualization

args.consecutive = True # if not consecutive, we include the sublist
args.undir_metric = False # if the metric is undirection so that we can accelerate the compute
args.lca_with_reoccur = False # if we count re-occurence of the longest common actions

args.exec_based = False # use the execution of the plans, instead of the pure plans
args.init_exec = False # use the input as the initial condition of the plans

args.partial_plan_gen = False # convert the demos into partial problems generated by the partial plans
args.goal_infer_with_on = False # if we ensure the generated plan goals are with "on"

args.object_centric_parse = False # parse the actions by objects
args.object_centric_affordance = False # when parsing the actions, assume different locations with objects
args.object_relevance_parse = False # parse the actions by objects, add actions from relevant objects

args.score_conv = "mean-max" # max-max denotes  max(max(per object action series similarity))
args.score_conv_mode = "weighted" # if weighted, the overall common_len will be weighed by the common len of each
args.heatmap = "mean" # type of heatmap

args.use_prompt = False # use prompt during embedding
args.plan_sim = "gecko" # token or embedding model names: [gecho,retsim,uni,unil,st5,st5l,st53b]
args.client = "goldfish" # m, xxs_it, m_it, m_it_bulk, xl_it, goldfish, goldfish_s, s_flash
args.num_demo = 1 # test: 0,1,4,8

args.icl_sim = "plan" # use random, plan, input similarity to sample the icl examples
args.icl_sim_mode = "static" # static, dynamic,reverse if dynamic, we sample demostration with distribution, num_demo become the std offset
args.natural_task = "trip" # trip, calendar

args.cluster_size = 8 # how many examples per cluster to search
args.per_cluster_sample_max = 2 # how examples per cluster to smaple

args.debug_mode = False


MAX_RETRIES = 5  # Maximum number of retries
RETRY_INTERVAL = 10  # Seconds between retries

embedder_map = {
    "token":None,
    # "retsim":tm.models.RetsimEmbedder(),
    "gecko":tm.models.GeckoEmbedder(),
    # "uni":tm.models.UnivSentEnc(),
    # "unil":tm.models.UnivSentEncLarge(),
    # "st5":tm.models.ST5Embedder(),
    # "st5l":tm.models.ST5Embedder_large(),
    # "st53b":tm.models.ST5Embedder_3b(),
}


# Usable: GEMINI_M, GEMINI_XXS_IT, GEMINI_M_IT (has rate limit), GEMINI_M_IT_BULK, GEMINI_XL_IT, GEMINI_GOLDFISH(1.5 pro), GEMINI_GOLDFISH_SHORT, GEMINI_S_FLASH
# model_url = model_client.ModelEntry.GEMINI_M_IT # @param {isTemplate: true}
# check this page for more info: https://g3doc.corp.google.com/learning/deepmind/evergreen/g3doc/shared_servers.md?cl=head#fair-use-policy

model_url_map = {
    "m": model_client.ModelEntry.GEMINI_M,
    "xxs_it": model_client.ModelEntry.GEMINI_XXS_IT,
    "m_it": model_client.ModelEntry.GEMINI_M_IT,
    "m_it_bulk": model_client.ModelEntry.GEMINI_M_IT_BULK,
    "xl_it": model_client.ModelEntry.GEMINI_XL_IT,
    "goldfish": model_client.ModelEntry.GEMINI_GOLDFISH,
    "goldfish_s": model_client.ModelEntry.GEMINI_GOLDFISH_SHORT,
    "s_flash": model_client.ModelEntry.GEMINI_S_FLASH,
}

em = embedder_map[args.plan_sim]
client = model_client.ModelClient(
    model_url= model_url_map[args.client],
    route = model_client.Route.LM_PROXY,
    # Enables automatic retries, ideal for
    # batch traffic, see go/lm-proxy-quota for more info.
    stub_options = model_client.BULK_STUB_OPTIONS,
)

# client.embed("it is a lovely day")

In [ ]:
print(data_lines[0])

{'num_cities': 3, 'cities': 'Tallinn**Berlin**Venice', 'constraints': 'Venice', 'durations': '2**4**3', 'total_days': '7', 'task': 'You plan to visit 3 European cities for 7 days in total. You only take direct flights to commute between cities. You want to spend 4 days in Berlin. You would like to visit Venice for 3 days. You are going to attend a wedding in Venice between day 5 and day 7. You want to spend 2 days in Tallinn.\n\nHere are the cities that have direct flights:\nBerlin and Venice, Tallinn and Berlin.\n\nFind a trip plan of visiting the cities for 7 days by taking direct flights to commute between them.', 'split': 'Train', 'solution': 'Here is the trip plan for visiting the 3 European cities for 7 days:\n\n**Day 1-2:** Arriving in Tallinn and visit Tallinn for 2 days.\n**Day 2:** Fly from Tallinn to Berlin.\n**Day 2-5:** Visit Berlin for 4 days.\n**Day 5:** Fly from Berlin to Venice.\n**Day 5-7:** Visit Venice for 3 days.'}


In [ ]:
# plan similarity

# pair_sim_mapping, pair_sim_interval = find_trip_pair_sim_mapping_main(data_lines, test_lines, args)


# ASR, for trip only
filename = "dev-300/trip/dev-300-1300-as-gecko-goldfish-20-random-static.json"

with gfile.Open('/cns/od-d/home/xinranzhao/'+filename, 'r') as f:
  output_collection = json.load(f)

asr_test_lines = []
for item in output_collection:
  temp = {"solution": item[-1]}
  asr_test_lines.append(temp)

for i in range(len(output_collection)):
  if i < len(output_collection):
    continue
  else:
    asr_test_lines.append({"solution": "beyond reach"})


pair_sim_mapping_asr, pair_sim_interval = find_trip_pair_sim_mapping_main(data_lines, asr_test_lines, args)


  0%|          | 0/300 [00:00<?, ?it/s]

In [ ]:
for icl_sim in ["asr"]:
  print("icl_sim:", icl_sim)

  args.icl_sim = icl_sim
  # pair_sim_map, pair_sim_interval, pair_sim_mapping_input = None, None, None

  if args.icl_sim == "plan":
    pair_sim_map = pair_sim_mapping
  elif args.icl_sim == "input":
    pair_sim_map = pair_sim_mapping_input
  elif args.icl_sim == "asr":
    pair_sim_map = pair_sim_mapping_asr

  if not args.partial_plan_gen:
    pair_sim_interval = []

  # for num_demo in [4,3,2,1]:
  for num_demo in [10,20,40,100]:
    args.num_demo = num_demo
    output_collection = trip_icl_main(data_lines, test_lines, args, pair_sim_map, pair_sim_interval, client)

    # new_output_collection = post_gen_clean(output_collection)

    # print(new_output_collection[0][-1])
    if args.exec_based:
      plan_sim = "es"
      if args.init_exec:
        plan_sim += "i"
      else:
        plan_sim += "v"
    else:
      plan_sim = "as"

    if args.object_centric_parse:
      plan_sim = "o"+plan_sim

    if args.partial_plan_gen:
      plan_sim = plan_sim+"-ppgid"

    segs = ["dev",str(args.test_scope),str(args.data_scope),plan_sim, args.plan_sim,args.client,str(args.num_demo),args.icl_sim,args.icl_sim_mode]

    filename = "-".join(segs)+"-iter1.json"

    with gfile.Open('/cns/od-d/home/xinranzhao/dev-300/trip/'+filename, 'w') as f:
      json.dump(output_collection, f)

icl_sim: asr


  0%|          | 0/300 [00:00<?, ?it/s]

Valid: 122
Correct: 122
Total: 300
City Stats: {9: [4, 4, 41], 10: [1, 1, 36], 5: [18, 18, 30], 4: [26, 26, 36], 7: [13, 13, 37], 8: [6, 6, 41], 3: [28, 28, 42], 6: [26, 26, 37]}


  0%|          | 0/300 [00:00<?, ?it/s]

Valid: 135
Correct: 135
Total: 300
City Stats: {9: [6, 6, 41], 10: [2, 2, 36], 5: [18, 18, 30], 4: [27, 27, 36], 7: [12, 12, 37], 8: [9, 9, 41], 3: [34, 34, 42], 6: [27, 27, 37]}


  0%|          | 0/300 [00:00<?, ?it/s]

Valid: 135
Correct: 135
Total: 300
City Stats: {9: [7, 7, 41], 10: [2, 2, 36], 5: [23, 23, 30], 4: [31, 31, 36], 7: [10, 10, 37], 8: [7, 7, 41], 3: [30, 30, 42], 6: [25, 25, 37]}


  0%|          | 0/300 [00:00<?, ?it/s]

Valid: 154
Correct: 154
Total: 300
City Stats: {9: [6, 6, 41], 10: [5, 5, 36], 5: [23, 23, 30], 4: [33, 33, 36], 7: [17, 17, 37], 8: [13, 13, 41], 3: [29, 29, 42], 6: [28, 28, 37]}
